# socialmedia_analytics
group : 6

target : Rotten Tomatoes

last_modification : 

---

## Top Box Office


## Coming Soon


## detail

TOMATOMETER

AUDIENCE SCORE

MOVIE INFO

```
Rating:	PG-13 (Some Fantasy Action/Violence)
Genre:	Adventure, Fantasy
Original Language:	English
Director:	David Yates
Producer:	David Heyman, Steve Kloves, Tim Lewis, J.K. Rowling, Lionel Wigram
Writer:	J.K. Rowling, Steve Kloves
Release Date (Theaters):	Apr 15, 2022  Wide
Box Office (Gross USA):	$42.2M
Runtime:	2h 22m
Distributor:	Warner Bros. Pictures
Sound Mix:	Dolby Digital, DTS, Dolby Atmos
View the collection:	Harry Potter
```

CAST & CREW
```
Eddie Redmayne
Newt Scamander
 
Jude Law
Albus Dumbledore
 
Mads Mikkelsen
Gellert Grindelwald
 
Ezra Miller
Credence Barebone
 
Dan Fogler
Jacob Kowalski
 
Alison Sudol
Queenie Goldstein
```

In [33]:
import os
import time
import datetime
import re
import pandas as pd


import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import chromedriver_autoinstaller
from bs4 import BeautifulSoup


startTime = time.time()
currentTime = datetime.datetime.today()
print('==============================\n')
print(f'Current time is {currentTime}')


fileDirectory = os.getcwd()
parentDirectory = os.getcwd()


class Website:
    def __init__(self, name, title, main_url, target_url):
        self.name = name
        self.title = title
        self.main_url = main_url
        self.target_url = target_url
        
    def info(self):
        print(f'Website.name : {self.name}')
        print(f'Website.title : {self.title}')
        print(f'Website.main_url : {self.main_url}')
        print(f'Website.target_url : {self.target_url}')


class WebElement:
    def __init__(self, name, type, identifier):
        self.name = name
        self.type = type
        self.identifier = identifier
        
    def info(self):
        print(f'WebElement.name : {self.name}')
        print(f'WebElement.type : {self.type}')
        print(f'WebElement.identifier : {self.identifier}')


class Preprocessor:
    def __init__(self) -> None:
        print('class Preprocessor : init')
        
    def clearText(self, texts):
        list_return = [text.text for text in texts]
        texts_cleared = [text.strip() for text in list_return]
        return texts_cleared

    def extractImage(self, contents):
        images_extracted = [tag['src'] for tag in contents]
        return images_extracted

    def extractURL(self, contents):
        url_extracted = [tag['href'] for tag in contents]
        return url_extracted
    
    def extractURLfromParent(self, contents):
        url_extracted = [tag.parent['href'] for tag in contents]
        return url_extracted

    def insertTextInURLs(self, text, location, urls):
        urls_inserted = []
        for url in urls:
            url_splited = url.split(location)
            url_inserted = url_splited[0] + location + text + url_splited[1]
            urls_inserted.append(url_inserted)
        return urls_inserted

    def preprocessContents(self, website, type, contents):
        if type == 'text':
            contents_preprocessed = self.clearText(contents)
        elif type == 'image':
            contents_preprocessed = self.extractImage(contents)
        elif type == 'url':
            if website.title == 'genre_top_100':
                contents_preprocessed = self.extractURLfromParent(contents)
            else:
                contents_preprocessed = self.extractURL(contents)
            if website.name == '블랙보드':
                contents_preprocessed = self.insertTextInURLs('courses','ultra/',contents_preprocessed)
            elif website.name == '고려대학교 경영대학':
                contents_preprocessed = [website.url.replace('notice.html','') + content for content in contents_preprocessed]
        return contents_preprocessed


class ActionChain:
    def __init__(self):
        print('class ActionChain : init')
        
    def waitAction(self, driver, action, maxWaitTime=30):
        try:
            if action.type == 'css':
                element = WebDriverWait(driver, maxWaitTime).until(expected_conditions.presence_of_element_located((By.CSS_SELECTOR, action.identifier)))
            elif action.type == 'xpath':
                element = WebDriverWait(driver, maxWaitTime).until(expected_conditions.presence_of_element_located((By.XPATH, action.identifier)))
        except Exception as e:
            print('ActionChain.waitAction error :', e)

    def clickAction(self, driver, action):
        chains = webdriver.ActionChains(driver)
        if action.type == 'css':
            button = driver.find_element(By.CSS_SELECTOR, action.identifier)
        elif action.type == 'xpath':
            button = driver.find_element(By.XPATH, action.identifier)
        chains.move_to_element(button).click().perform()
        # chains.context_click(button).perform()

    def inputAction(self, driver, action):
        if action.type == 'css':
            inputElement = driver.find_element(By.CSS_SELECTOR, action.identifier)
        if action.type == 'xpath':
            inputElement = driver.find_element(By.XPATH, action.identifier)
        text =re.findall('\(([^)]+)', action.name)
        inputElement.send_keys(text)

    def enterAction(self, driver, action):
        if action.type == 'css':
            keyElement = driver.find_element(By.CSS_SELECTOR, action.identifier)
        if action.type == 'xpath':
            keyElement = driver.find_element(By.XPATH, action.identifier)
        keyElement.send_keys(Keys.ENTER)

    def doActionChain(self, driver, actions):
        for i, action in enumerate(actions):
            if 'wait' in action.name:
                self.waitAction(driver, action)
                print(f'Step {i} : {action.name} done')
            elif 'time' in action.name:
                driver.implicitly_wait(5)
                print(f'Step {i} : {action.name} done')
            elif 'click' in action.name:
                self.clickAction(driver, action)
                print(f'Step {i} : {action.name} done')
            elif 'input' in action.name:
                self.inputAction(driver, action)
                print(f'Step {i} : {action.name} done')
            elif 'enter' in action.name:
                self.enterAction(driver, action)
                print(f'Step {i} : {action.name} done')
            elif 'back' in action.name:
                driver.back()
                print(f'Step {i} : {action.name} done')
            else:
                print('')
            
    def doActionRoutine(self, driver, iteration, iteration_actions):
        for i, action in enumerate(iteration_actions):
            if 'wait' in action.name:
                self.waitAction(driver, action)
                # print(f'Step {i} : {action.name} done')
            elif 'load' in action.name:
                driver.implicitly_wait(5)
                # print(f'Step {i} : {action.name} done')
            elif 'time' in action.name:
                time.sleep(3)
                # print(f'Step {i} : {action.name} done')
            elif 'click' in action.name:
                webdriver.ActionChains(driver).move_to_element(iteration).click().perform()
                # print(f'Step {i} : {action.name} done')
            elif 'input' in action.name:
                self.inputAction(driver, action)
                # print(f'Step {i} : {action.name} done')
            elif 'enter' in action.name:
                self.enterAction(driver, action)
                # print(f'Step {i} : {action.name} done')
            elif 'back' in action.name:
                driver.back()
                # print(f'Step {i} : {action.name} done')
            else:
                print('')


class WebDriverUpdateChecker:
    def __init__(self):
        print('class WebDriverUpdateChecker : init')
    
    def checkWebdriverVersion(self):
        chromeVersion = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
        parentDirectoryList = os.listdir(parentDirectory)
        if 'chromedriver' not in parentDirectoryList:
            os.mkdir(parentDirectory + '/chromedriver')
        chromedriverDirectory = parentDirectory + '/chromedriver'
        chromedriverDirectoryList = os.listdir(chromedriverDirectory)
        if chromeVersion not in chromedriverDirectoryList:
            chromedriver_autoinstaller.install(False, chromedriverDirectory)
            print('\ncheckWebdriverVersion : downloaded the latest chromedriver')
        else:
            print('\ncheckWebdriverVersion : already the latest chromedriver')

    def getWebdriverDirectory(self):
        chromeVersion = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
        webdriverDirectory = parentDirectory + '/chromedriver' + '/' + chromeVersion + '/' + 'chromedriver'
        return webdriverDirectory


class Crawler(WebDriverUpdateChecker, ActionChain, Preprocessor):
    def __init__(self):
        super().__init__()
        print('\tclass Crawler : init')
    
    def getResponse(self, url):
        try:
            response = requests.get(url)
        except Exception as e:
            print('Crawler.getResponse error : ', e)
        return response

    def setChromeOptions(self, headless=True):
        chromeOptions = webdriver.ChromeOptions()
        if headless:
            chromeOptions.add_argument('headless')
        chromeOptions.add_argument('lang=ko_KR')
        chromeOptions.add_argument('disable-gpu')
        chromeOptions.add_argument('window-size=1920x1080')
        chromeOptions.add_argument(
            'user-agent='+'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Safari/537.36')
        return chromeOptions

    def getDriver(self, url, actions):
        super().checkWebdriverVersion()
        webdriverDirectory = super().getWebdriverDirectory()
        chromeOptions = self.setChromeOptions()
        driver = webdriver.Chrome(webdriverDirectory, options=chromeOptions)
        try:
            driver.get(url)
        except Exception as e:
            print('Crawling.getDriver error : ', e)
        if actions is not None:
            super().doActionChain(driver, actions)
        return driver

    def getSource(self, url, actions, module):
        if module == 'requests':
            reponse = self.getResponse(url)
            source = reponse.content
        elif module == 'selenium':
            driver = self.getDriver(url, actions)
            source = driver.page_source
        return source

    def parseSource(self, source, parser):
        soup = BeautifulSoup(source, parser)
        return soup

    def selectText(self, soup, content):
        if soup is not None:
            try:
                texts = soup.select(content.identifier)
            except Exception as e:
                print('Crawler.selectText error : ', e)
            return texts

    def selectTable(self, soup, content):
        if soup is not None:
            try:
                target = soup.select(content.identifier)
            except Exception as e:
                print('Crawler.selectTable error : ', e)
            table_list = pd.read_html(str(target))
            return table_list

    def selectImage(self, soup, content):
        if soup is not None:
            try:
                images = soup.find_all('img')
            except Exception as e:
                print('Crawler.selectText error : ', e)
            return images

    def getContents(self, website, contents, actions=None, module='selenium', parser='lxml'):
        source = self.getSource(website.target_url, actions, module)
        soup = self.parseSource(source, parser)
        dict_return = {}
        for content in contents:
            if 'text' in content.name:
                key = re.findall('\(([^)]+)', content.name)
                texts = self.selectText(soup, content)
                texts_preprocessed = super().preprocessContents(website, 'text', texts)
                dict_return[key[0]] = texts_preprocessed
            elif 'table' in content.name:
                key = re.findall('\(([^)]+)', content.name)
                table = self.selectTable(soup, content)
                dict_return[key[0]] = table[0]
            elif 'image' in content.name:
                key = re.findall('\(([^)]+)', content.name)
                images = self.selectImage(soup, content)
                images_preprocessed = super().preprocessContents(website, 'image', images)
                dict_return[key[0]] = images_preprocessed
            elif 'url' in content.name:
                key = re.findall('\(([^)]+)', content.name)
                urls = self.selectText(soup, content)
                urls_preprocessed = super().preprocessContents(website, 'url', urls)
                dict_return[key[0]] = urls_preprocessed
            else:
                print('')
        return dict_return
    
    
    
    
    def getContentsInSequence(self, website, contents, iteration, iteration_actions_in, iteration_actions_out, actions=None, module='selenium', parser='lxml'):
        driver = self.getDriver(website.target_url, actions)
        chains = webdriver.ActionChains(driver)
        
        if iteration.type == 'css':
            iterations = driver.find_elements(By.CSS_SELECTOR, iteration.identifier)
        elif iteration.type == 'xpath':
            iterations = driver.find_elements(By.XPATH, iteration.identifier)
        keys = [a.text for a in iterations]
        
        iteration_dict = {}
        # for i in range(len(iterations)):
        for i in range(4):
            
            if iteration.type == 'css':
                iterations = driver.find_elements(By.CSS_SELECTOR, iteration.identifier)
            elif iteration.type == 'xpath':
                iterations = driver.find_elements(By.XPATH, iteration.identifier)

            super().doActionRoutine(driver, iterations[i], iteration_actions_in)
            
            source = driver.page_source
            soup = self.parseSource(source, parser)
            dict_return = {}
            for content in contents:
                if 'text' in content.name:
                    key = re.findall('\(([^)]+)', content.name)
                    texts = self.selectText(soup, content)
                    texts_preprocessed = super().preprocessContents(website, 'text', texts)
                    dict_return[key[0]] = texts_preprocessed
                elif 'table' in content.name:
                    key = re.findall('\(([^)]+)', content.name)
                    table = self.selectTable(soup, content)
                    dict_return[key[0]] = table[0]
                elif 'image' in content.name:
                    key = re.findall('\(([^)]+)', content.name)
                    images = self.selectImage(soup, content)
                    images_preprocessed = super().preprocessContents(website, 'image', images)
                    dict_return[key[0]] = images_preprocessed
                elif 'url' in content.name:
                    key = re.findall('\(([^)]+)', content.name)
                    urls = self.selectText(soup, content)
                    urls_preprocessed = super().preprocessContents(website, 'url', urls)
                    dict_return[key[0]] = urls_preprocessed
                else:
                    print('')
            iteration_dict[keys[i]] = dict_return
            super().doActionRoutine(driver, iterations[i], iteration_actions_out)
            print(f'{website.name} - {i}/{len(iterations)} {keys[i]} Done!                                   ', end='\r')
        return iteration_dict

    
    
crawler = Crawler()

endTime = time.time()
print(f'\nRunning time : {endTime - startTime} \n')
print('==============================\n\n\n')


Current time is 2022-04-24 08:42:09.105620
class WebDriverUpdateChecker : init
	class Crawler : init

Running time : 0.01458120346069336 






In [42]:
rotten_tomatoes = Website('rotten tomatoes', 'genre_top_100', 'https://www.rottentomatoes.com', 'https://www.rottentomatoes.com/top')
contents = [
    WebElement('text(genre_names)', 'css', '#main_container > div.container-masonry > div:nth-child(3) > section > div > ul > li > a > div'),
    WebElement('url(genre_urls)', 'css', '#main_container > div.container-masonry > div:nth-child(3) > section > div > ul > li > a > div')
]
actions = []
genre_top_100_crawled = crawler.getContents(rotten_tomatoes, contents, module='requests')

{'genre_names': ['Top 100 Action & Adventure Movies',
  'Top 100 Animation Movies',
  'Top 100 Art House & International Movies',
  'Top 100 Classics Movies',
  'Top 100 Comedy Movies',
  'Top 100 Documentary Movies',
  'Top 100 Drama Movies',
  'Top 100 Horror Movies',
  'Top 100 Kids & Family Movies',
  'Top 100 Musical & Performing Arts Movies',
  'Top 100 Mystery & Suspense Movies',
  'Top 100 Romance Movies',
  'Top 100 Science Fiction & Fantasy Movies',
  'Top 100 Special Interest Movies',
  'Top 100 Sports & Fitness Movies',
  'Top 100 Television Movies',
  'Top 100 Western Movies'],
 'genre_urls': ['/top/bestofrt/top_100_action__adventure_movies/',
  '/top/bestofrt/top_100_animation_movies/',
  '/top/bestofrt/top_100_art_house__international_movies/',
  '/top/bestofrt/top_100_classics_movies/',
  '/top/bestofrt/top_100_comedy_movies/',
  '/top/bestofrt/top_100_documentary_movies/',
  '/top/bestofrt/top_100_drama_movies/',
  '/top/bestofrt/top_100_horror_movies/',
  '/top/bestof

In [34]:
startTime = time.time()
currentTime = datetime.datetime.today()
print('==============================\n')
print(f'Current time is {currentTime}')


website_list = []
for i, genre_names in enumerate(genre_top_100_crawled['genre_names']):
    target_url = rotten_tomatoes.main_url + genre_top_100_crawled['genre_urls'][i]
    website_list.append(Website(genre_names, 'top_movies', 'https://www.rottentomatoes.com', target_url))

    
movies_dict = {}
for target_website in website_list[:1]:
    iteration = WebElement('iterations','css','#top_movies_main > div > table > tbody > tr > td:nth-child(3) > a')
    iteration_actions_in = [
        WebElement('click','',''),
        WebElement('load','','')
    ]
    iteration_actions_out = [
        WebElement('back','',''),
        WebElement('time','','')
    ]
    contents = [
        # WebElement('text(TOMATOMETER)', 'xpath', '//*[@id="topSection"]/div[1]/score-board//div/div[2]/div[1]/div/score-icon-critic//div/span[2]'),
        # WebElement('text(AUDIENCE_SCORE)', 'css', 'div > span.percentage'),
        WebElement('text(detail_label)', 'css', '#mainColumn > section.panel.panel-rt.panel-box.movie_info.media > div > div > ul > li > div.meta-label.subtle'),
        WebElement('text(detail_info)', 'css', '#mainColumn > section.panel.panel-rt.panel-box.movie_info.media > div > div > ul > li > div.meta-value'),
        WebElement('text(detail_crew)', 'css', '#movie-cast > div > div > div > div.media-body > a > span'),
        # WebElement('text(critic_reivews)', 'css', 'div.review-quote > div > p'),
        WebElement('text(audience_reivews)', 'css', '#audience_reviews > ul > li > div.mop-audience-reviews__review-quote > div.mop-audience-reviews__review--comment.clamp.clamp-4.js-clamp')
    ]
    movies_top_100_dict = crawler.getContentsInSequence(target_website, contents, iteration, iteration_actions_in, iteration_actions_out)
    movies_dict[target_website.name] = movies_top_100_dict
    print(f'{target_website.name} -  Done!                                   ', end='\n\n')
    
    
endTime = time.time()
print(f'\nRunning time : {endTime - startTime} \n')
print('==============================\n\n\n')


Current time is 2022-04-24 08:42:12.017051

checkWebdriverVersion : already the latest chromedriver
Top 100 Action & Adventure Movies -  Done!                                                                             


Running time : 75.7262442111969 






In [35]:
movies_dict

{'Top 100 Action & Adventure Movies': {'Black Panther (2018)': {'TOMATOMETER': [],
   'AUDIENCE_SCORE': [],
   'detail_label': ['Rating:',
    'Genre:',
    'Original Language:',
    'Director:',
    'Producer:',
    'Writer:',
    'Release Date (Theaters):',
    'Release Date (Streaming):',
    'Box Office (Gross USA):',
    'Runtime:',
    'Distributor:',
    'Sound Mix:',
    'Aspect Ratio:',
    'View the collection:'],
   'detail_info': ['PG-13 (Sequences of Action Violence|A Brief Rude Gesture)',
    'Adventure, \n                        \n                        Fantasy, \n                        \n                        Action',
    'English',
    'Ryan Coogler',
    'Kevin Feige',
    'Ryan Coogler, \n                            \n                        \n                            \n                                Joe Robert Cole',
    'Feb 16, 2018\n\xa0wide',
    'May 2, 2018',
    '$700.2M',
    '2h 14m',
    'Walt Disney',
    'DTS, Dolby Atmos',
    'Scope (2.35:1)',


In [45]:
columns = ['Rating', 'Genre', 'Original_Language', 'Director', 'Producer', 'Writer', 'Release Date(Theaters)', 'Release Date (Streaming)', 'Box Office (Gross USA)', 'Runtime', 'Distributor', 'Sound Mix', 'Aspect Ratio', 'View the collection']

for key, value_dict in movies_dict.items():
    sheet_title = 'df_' + key.replace(' ', '_').replace('&','and')
    context = []
    for key, value in value_dict.items():
        movie_title = key
        movie_info = value
        context.append([variable.replace('\n','').replace('  ','').replace('\xa0',' ') for variable in movie_info['detail_info']])
        dict(zip(aaa, bbb))
        
    sheet_title = pd.DataFrame(data=context, index=value_dict.keys(), columns=columns)
    


df_Top_100_Action_and_Adventure_Movies
df_Top_100_Animation_Movies
df_Top_100_Art_House_and_International_Movies


ValueError: 14 columns passed, passed data had 13 columns

In [1]:
movies_dict = {'Top 100 Action & Adventure Movies': {'Black Panther (2018)': {'detail_info': ['PG-13 (Sequences of Action Violence|A Brief Rude Gesture)',
    'Adventure, \n                        \n                        Fantasy, \n                        \n                        Action',
    'English',
    'Ryan Coogler',
    'Kevin Feige',
    'Ryan Coogler, \n                            \n                        \n                            \n                                Joe Robert Cole',
    'Feb 16, 2018\n\xa0wide',
    'May 2, 2018',
    '$700.2M',
    '2h 14m',
    'Walt Disney',
    'DTS, Dolby Atmos',
    'Scope (2.35:1)',
    'Marvel Cinematic Universe'],
   'detail_crew': ['Chadwick Boseman',
    'Michael B. Jordan',
    "Lupita Nyong'o",
    'Danai Gurira',
    'Martin Freeman',
    'Daniel Kaluuya',
    'Letitia Wright',
    'Winston Duke',
    'Angela Bassett',
    'Forest Whitaker',
    'Andy Serkis',
    'Florence Kasumba',
    'Sterling K. Brown',
    'John Kani',
    'Ryan Coogler',
    'Ryan Coogler',
    'Joe Robert Cole',
    'Kevin Feige',
    "Louis D'Esposito",
    'Victoria Alonso',
    'Nate Moore',
    'Jeffrey Chernov',
    'Stan Lee',
    'Rachel Morrison',
    'Debbie Berman',
    'Michael P. Shawver',
    'Ludwig Göransson',
    'Hannah Beachler',
    'Ruth E. Carter',
    'Sarah Halley Finn']},
  'Avengers: Endgame (2019)': {'detail_info': ['PG-13 (Sequences of Sci-Fi Violence|Action|Some Language)',
    'Sci-fi, \n                        \n                        Adventure, \n                        \n                        Fantasy, \n                        \n                        Action',
    'English',
    'Anthony Russo, \n                            \n                        \n                            \n                                Joe Russo',
    'Kevin Feige',
    'Christopher Markus, \n                            \n                        \n                            \n                                Stephen McFeely',
    'Apr 26, 2019\n\xa0wide',
    'Jul 30, 2019',
    '$858.4M',
    '3h 1m',
    'Walt Disney',
    'Dolby Atmos, DTS, Dolby Digital, SDDS',
    'Scope (2.35:1)',
    'Marvel Cinematic Universe'],
   'detail_crew': ['Robert Downey Jr.',
    'Chris Evans',
    'Mark Ruffalo',
    'Chris Hemsworth',
    'Scarlett Johansson',
    'Jeremy Renner',
    'Brie Larson',
    'Paul Rudd',
    'Don Cheadle',
    'Karen Gillan',
    'Danai Gurira',
    'Bradley Cooper',
    'Gwyneth Paltrow',
    'Jon Favreau',
    'Benedict Wong',
    'Tessa Thompson',
    'Josh Brolin',
    'Tilda Swinton',
    'Robert Redford',
    'Dave Bautista',
    'Anthony Russo',
    'Joe Russo',
    'Christopher Markus',
    'Stephen McFeely',
    'Kevin Feige',
    "Louis D'Esposito",
    'Victoria Alonso',
    'Michael Grillo',
    'Trinh Tran',
    'Jon Favreau',
    'James Gunn',
    'Stan Lee',
    'Trent Opaloch',
    'Jeffrey Ford',
    'Matthew Schmidt',
    'Alan Silvestri',
    'Charles Wood',
    'Judianna Makovsky',
    'Sarah Halley Finn']},
  'Mission: Impossible - Fallout (2018)': {'detail_info': ['PG-13 (Intense Sequences of Action|Brief Strong Language|Violence)',
    'Mystery & thriller, \n                        \n                        Adventure, \n                        \n                        Action',
    'English',
    'Christopher McQuarrie',
    'Tom Cruise, \n                            \n                        \n                            \n                                Christopher McQuarrie, \n                            \n                        \n                            \n                                Jake Myers, \n                            \n                        \n                            \n                                J.J. Abrams',
    'Christopher McQuarrie',
    'Jul 27, 2018\n\xa0wide',
    'Oct 23, 2018',
    '$220.1M',
    '2h 27m',
    'Paramount Pictures',
    'DTS, Dolby Atmos, Dolby Digital',
    'Scope (2.35:1)'],
   'detail_crew': ['Tom Cruise',
    'Henry Cavill',
    'Ving Rhames',
    'Simon Pegg',
    'Rebecca Ferguson',
    'Sean Harris',
    'Angela Bassett',
    'Michelle Monaghan',
    'Alec Baldwin',
    'Vanessa Kirby',
    'Wes Bentley',
    'Frederick Schmidt',
    'Wolf Blitzer',
    'Christopher McQuarrie',
    'Christopher McQuarrie',
    'David Ellison',
    'Dana Goldberg',
    'Don Granger',
    'Tom Cruise',
    'Christopher McQuarrie',
    'Jake Myers',
    'J.J. Abrams',
    'Rob Hardy',
    'Eddie Hamilton',
    'Lorne Balfe',
    'Peter Wenham',
    'Jeffrey Kurland']},
  'Mad Max: Fury Road (2015)': {'detail_info': ['R (Intense Sequences of Violence|Disturbing Images)',
    'Adventure, \n                        \n                        Action',
    'English',
    'George Miller',
    'Doug Mitchell, \n                            \n                        \n                            \n                                George Miller, \n                            \n                        \n                            \n                                P.J. Voeten',
    'George Miller, \n                            \n                        \n                            \n                                Brendan McCarthy, \n                            \n                        \n                            \n                                Nick Lathouris',
    'May 15, 2015\n\xa0wide',
    'Sep 1, 2015',
    '$153.6M',
    '2h 0m',
    'Warner Bros. Pictures',
    'Dolby Atmos, Dolby Digital, SDDS, Datasat',
    'Scope (2.35:1)'],
   'detail_crew': ['Tom Hardy',
    'Charlize Theron',
    'Nicholas Hoult',
    'Hugh Keays-Byrne',
    'Josh Helman',
    'Nathan Jones',
    'Zoë Kravitz',
    'Rosie Huntington-Whiteley',
    'Riley Keough',
    'Abbey Lee',
    'Courtney Eaton',
    'John Howard',
    'Richard Carter',
    'iOTA',
    'Angus Sampson',
    'Jennifer Hagan',
    'Coco Jack Gillies',
    'George Miller',
    'George Miller',
    'Brendan McCarthy',
    'Nick Lathouris',
    'Doug Mitchell',
    'George Miller',
    'P.J. Voeten',
    'Iain Smith',
    'Christopher DeFaria',
    'Courtenay Valenti',
    'Graham Burke',
    'Bruce Berman',
    'Steven Mnuchin',
    'John Seale',
    'Margaret Sixel',
    'Junkie XL',
    'Colin Gibson',
    'Jenny Beavan',
    'Ronna Kress',
    'Nikki Barrett']}},
 'Top 100 Animation Movies': {'Toy Story 4 (2019)': {'detail_info': ['G',
    'Comedy, \n                        \n                        Kids & family, \n                        \n                        Adventure, \n                        \n                        Fantasy, \n                        \n                        Animation',
    'English',
    'Josh Cooley',
    'Mark Nielsen, \n                            \n                        \n                            \n                                Jonas Rivera, \n                            \n                        \n                            \n                                Galyn Susman',
    'Andrew Stanton, \n                            \n                        \n                            \n                                Stephany Folsom',
    'Jun 21, 2019\n\xa0wide',
    'Oct 1, 2019',
    '$434.0M',
    '1h 40m',
    'Walt Disney',
    'DTS, Dolby Digital, Dolby Atmos',
    'Scope (2.35:1)',
    'Pixar'],
   'detail_crew': ['Tom Hanks',
    'Tim Allen',
    'Annie Potts',
    'Tony Hale',
    'Keegan-Michael Key',
    'Madeleine McGraw',
    'Christina Hendricks',
    'Jordan Peele',
    'Keanu Reeves',
    'Ally Maki',
    'Jay Hernandez',
    'Lori Alan',
    'Joan Cusack',
    'Bonnie Hunt',
    'Kristen Schaal',
    'Emily Davis',
    'Wallace Shawn',
    'John Ratzenberger',
    'Blake Clark',
    'June Squibb',
    'Josh Cooley',
    'Andrew Stanton',
    'Stephany Folsom',
    'Mark Nielsen',
    'Jonas Rivera',
    'Galyn Susman',
    'Andrew Stanton',
    'Lee Unkrich',
    'Pete Docter',
    'Patrick Lin',
    'Jean-Claude Kalache',
    'Axel Geddes',
    'Randy Newman',
    'Bob Pauley',
    'Kevin Reher',
    'Natalie Lyon']},
  'Spider-Man: Into the Spider-Verse (2018)': {'detail_info': ['PG (Mild Language|Frenetic Action Violence|Thematic Elements)',
    'Comedy, \n                        \n                        Kids & family, \n                        \n                        Adventure, \n                        \n                        Fantasy, \n                        \n                        Animation, \n                        \n                        Action',
    'English',
    'Bob Persichetti, \n                            \n                        \n                            \n                                Peter Ramsey, \n                            \n                        \n                            \n                                Rodney Rothman',
    'Avi Arad, \n                            \n                        \n                            \n                                Amy Pascal, \n                            \n                        \n                            \n                                Phil Lord, \n                            \n                        \n                            \n                                Chris Miller, \n                            \n                        \n                            \n                                Christina Steinberg',
    'Phil Lord, \n                            \n                        \n                            \n                                Rodney Rothman',
    'Dec 14, 2018\n\xa0wide',
    'Mar 19, 2019',
    '$190.2M',
    '1h 57m',
    'Sony Pictures Entertainment, \n                        \n                        Columbia Pictures',
    'Dolby Atmos, DTS, Dolby Digital, SDDS',
    'Scope (2.35:1)'],
   'detail_crew': ['Shameik Moore',
    'Jake Johnson',
    'Hailee Steinfeld',
    'Mahershala Ali',
    'Brian Tyree Henry',
    'Lily Tomlin',
    'Luna Lauren Velez',
    'Zoë Kravitz',
    'John Mulaney',
    'Kimiko Glenn',
    'Nicolas Cage',
    'Kathryn Hahn',
    'Liev Schreiber',
    'Chris Pine',
    'Natalie Morales',
    'Bob Persichetti',
    'Peter Ramsey',
    'Rodney Rothman',
    'Phil Lord',
    'Rodney Rothman',
    'Stan Lee',
    'Brian Michael Bendis',
    'Will Allegra',
    'Avi Arad',
    'Amy Pascal',
    'Phil Lord',
    'Chris Miller',
    'Christina Steinberg',
    'Robert Fisher Jr.',
    'Daniel Pemberton',
    'Justin K. Thompson',
    'Dean Gordon',
    "Patrick O'Keefe",
    'Mary Hidalgo']},
  'Inside Out (2015)': {'detail_info': ['PG (Some Action|Mild Thematic Elements)',
    'Animation, \n                        \n                        Kids & family, \n                        \n                        Fantasy, \n                        \n                        Comedy',
    'English',
    'Pete Docter',
    'Jonas Rivera',
    'Pete Docter, \n                            \n                        \n                            \n                                Meg LeFauve, \n                            \n                        \n                            \n                                Josh Cooley',
    'Jun 19, 2015\n\xa0wide',
    'Nov 3, 2015',
    '$2.0M',
    '1h 35m',
    'Walt Disney',
    'Dolby Atmos, Datasat',
    'Pixar'],
   'detail_crew': ['Amy Poehler',
    'Phyllis Smith',
    'Richard Kind',
    'Bill Hader',
    'Lewis Black',
    'Mindy Kaling',
    'Kaitlyn Dias',
    'Diane Lane',
    'Kyle MacLachlan',
    'Pete Docter',
    'Ronnie del Carmen',
    'Pete Docter',
    'Meg LeFauve',
    'Josh Cooley',
    'Jonas Rivera',
    'Patrick Lin',
    'Kevin Nolting',
    'Michael Giacchino',
    'Ralph Eggleston',
    'Kevin Reher',
    'Natalie Lyon']},
  'Coco (2017)': {'detail_info': ['PG (Thematic Elements)',
    'Music, \n                        \n                        Animation, \n                        \n                        Adventure, \n                        \n                        Comedy',
    'English',
    'Lee Unkrich',
    'Darla K. Anderson',
    'Adrian Molina, \n                            \n                        \n                            \n                                Matthew Aldrich',
    'Nov 22, 2017\n\xa0wide',
    'Feb 13, 2018',
    '$210.5M',
    '1h 49m',
    'Walt Disney',
    'Dolby Atmos',
    'Scope (2.35:1)',
    'Pixar'],
   'detail_crew': ['Anthony Gonzalez',
    'Gael García Bernal',
    'Benjamin Bratt',
    'Alanna Ubach',
    'Jaime Camil',
    'Alfonso Arau',
    'Sofía Espinosa',
    'Selene Luna',
    'Ana Ofelia Murguía',
    'Renee Victor',
    'Luis Valdez',
    'Herbert Siguenza',
    'Carla Medina',
    'Edward James Olmos',
    'Lombardo Boyer',
    'Gabriel Iglesias',
    'Cheech Marin',
    'Blanca Araceli',
    'Lee Unkrich',
    'Adrian Molina',
    'Adrian Molina',
    'Matthew Aldrich',
    'Darla K. Anderson',
    'John Lasseter',
    'Steve Bloom',
    'Lee Unkrich',
    'Michael Giacchino']}},
 'Top 100 Art House & International Movies': {'Parasite (Gisaengchung) (2019)': {'detail_info': ['R (Sexual Content|Language|Some Violence)',
    'Drama, \n                        \n                        Mystery & thriller, \n                        \n                        Comedy',
    'Korean',
    'Bong Joon Ho',
    'Kwak Sin-ae, \n                            \n                        \n                            \n                                Moon Yanggwon',
    'Bong Joon Ho, \n                            \n                        \n                            \n                                Han Jinwon',
    'Nov 1, 2019\n\xa0wide',
    'Oct 11, 2019',
    '$53.4M',
    '2h 12m',
    'Neon',
    'Dolby Atmos, Dolby Digital',
    'Scope (2.35:1)'],
   'detail_crew': ['Song Kang-ho',
    'Jo Yeo-jeong',
    'Park So-dam',
    'Choi Woo-sik',
    'Lee Sun-gyun',
    'Park Seo-joon',
    'Jung Ji-so',
    'Lee Jeong-eun',
    'Andreas Fronk',
    'Jang Hye-jin',
    'Park Myeong-hoon',
    'Lee Ji-hye',
    'Jung Hyun-jun',
    'Park Keun-rok',
    'Lee Joo-hyung',
    'Lee Dong-yong',
    'Bong Joon Ho',
    'Bong Joon Ho',
    'Han Jinwon',
    'Miky Lee',
    'Kwak Sin-ae',
    'Moon Yanggwon',
    'Hong Kyung-pyo',
    'Kim Chang-ho',
    'Yang Jin-mo',
    'Jung Jae-il',
    'Lee Ha-jun',
    'Choi Seyeon']},
  'The Cabinet of Dr. Caligari (Das Cabinet des Dr. Caligari) (1920)': {'detail_info': ['Horror, \n                        \n                        Fantasy, \n                        \n                        Mystery & thriller',
    'German',
    'Robert Wiene',
    'Rudolf Meinert, \n                            \n                        \n                            \n                                Erich Pommer',
    'Hans Janowitz, \n                            \n                        \n                            \n                                Carl Mayer',
    'Mar 19, 1921\n\xa0wide',
    'Feb 16, 1999',
    '1h 9m',
    'Grapevine Video, \n                        \n                        Kino Video'],
   'detail_crew': ['Werner Krauss',
    'Conrad Veidt',
    'Lil Dagover',
    'Friedrich Feher',
    'Rudolf Lettinger',
    'Hans Heinrich von Twardowski',
    'Rudolf Klein-Rogge',
    'Robert Wiene',
    'Hans Janowitz',
    'Carl Mayer',
    'Rudolf Meinert',
    'Erich Pommer',
    'Albert Band',
    'Giuseppe Becce',
    'Willy Hameister']},
  'Seven Samurai (Shichinin no Samurai) (1956)': {'detail_info': ['Action',
    'Japanese',
    'Akira Kurosawa',
    'Sojiro Motoki',
    'Shinobu Hashimoto, \n                            \n                        \n                            \n                                Akira Kurosawa, \n                            \n                        \n                            \n                                Hideo Oguni',
    'Nov 19, 1956\n\xa0wide',
    'Sep 5, 2006',
    '$271.7K',
    '3h 28m',
    'Columbia Pictures',
    'Mono',
    'Flat (1.37:1)'],
   'detail_crew': ['Toshiro Mifune',
    'Takashi Shimura',
    'Yoshio Inaba',
    'Seiji Miyaguchi',
    'Minoru Chiaki',
    'Daisuke Katô',
    'Isao Kimura',
    'Kunihori Kodo',
    'Kamatari Fujiwara',
    'Yoshio Tsuchiya',
    'Akira Kurosawa',
    'Shinobu Hashimoto',
    'Akira Kurosawa',
    'Hideo Oguni',
    'Sojiro Motoki',
    'Fumio Hayasaka',
    'Asakazu Nakai',
    'Akira Kurosawa',
    'Takashi Matsuyama',
    'Kôhei Ezaki',
    'Hiromichi Horikawa',
    'Fumio Yanoguchi',
    'Shigeru Mori']},
  'La Grande illusion (Grand Illusion) (1938)': {'detail_info': ['Drama, \n                        \n                        War',
    'French (Canada)',
    'Jean Renoir',
    'Albert Pinkovitch, \n                            \n                        \n                            \n                                Frank Rollmer',
    'Sep 12, 1937\n\xa0wide',
    'Nov 23, 1999',
    '$516.5K',
    '1h 57m',
    'Home Vision Entertainment, \n                        \n                        Barr Entertainment'],
   'detail_crew': ['Pierre Fresnay',
    'Erich von Stroheim',
    'Jean Gabin',
    'Marcel Dalio',
    'Dita Parlo',
    'Julien Carette',
    'Gaston Modot',
    'Georges Peclet',
    'Jean Dasté',
    'Sylvain Itkine',
    'Jacques Becker',
    'Werner Florian',
    'Jean Renoir',
    'Albert Pinkovitch',
    'Frank Rollmer',
    'Joseph Kosma',
    'Christian Matras',
    'Marthe Huguet',
    'Renée Lichtig',
    'Eugène Lourié',
    'René Decrais',
    'Raymond Blondy']}},
 'Top 100 Classics Movies': {'It Happened One Night (1934)': {'detail_info': ['Romance, \n                        \n                        Comedy',
    'English',
    'Frank Capra',
    'Frank Capra, \n                            \n                        \n                            \n                                Harry Cohn',
    'Samuel Hopkins Adams, \n                            \n                        \n                            \n                                Robert Riskin',
    'Feb 22, 1934\n\xa0original',
    'Dec 28, 1999',
    '1h 45m',
    'Columbia Pictures, \n                        \n                        Columbia TriStar Home Video, \n                        \n                        RCA/Columbia',
    'Mono',
    '35mm, Flat (1.37:1)'],
   'detail_crew': ['Claudette Colbert',
    'Clark Gable',
    'Walter Connolly',
    'Roscoe Karns',
    'Alan Hale',
    'Ward Bond',
    'Frank Capra',
    'Frank Capra',
    'Howard Jackson',
    'Samuel Hopkins Adams',
    'Robert Riskin',
    'Harry Cohn',
    'Frank Capra',
    'Harry Cohn',
    'Howard Jackson',
    'Louis Silvers',
    'Joseph Walker',
    'Gene Havlick',
    'Stephen Goosson',
    'Robert Kalloch']},
  'Citizen Kane (1941)': {'detail_info': ['PG',
    'Drama',
    'English',
    'Orson Welles',
    'Orson Welles',
    'Herman J. Mankiewicz, \n                            \n                        \n                            \n                                Orson Welles, \n                            \n                        \n                            \n                                John Houseman',
    'May 1, 1941\n\xa0wide',
    'Feb 23, 2010',
    '$977.3K',
    '1h 59m',
    'RKO Radio Pictures',
    'Mono',
    'Academy (1.33:1)'],
   'detail_crew': ['Orson Welles',
    'Joseph Cotten',
    'Ruth Warrick',
    'Everett Sloane',
    'Agnes Moorehead',
    'Dorothy Comingore',
    'Ray Collins',
    'George Coulouris',
    'William Alland',
    'Paul Stewart',
    'Erskine Sanford',
    'Orson Welles',
    'Herman J. Mankiewicz',
    'Orson Welles',
    'John Houseman',
    'Orson Welles',
    'Bernard Herrmann',
    'Gregg Toland',
    'Robert Wise',
    'Van Nest Polglase',
    'Perry Ferguson',
    'Darrell Silvera',
    'Edward Stevenson',
    'Maurice Seiderman',
    'John Aalberg',
    'Russell A. Cully',
    'Vernon L. Walker']},
  'The Wizard of Oz (1939)': {'detail_info': ['G',
    'Fantasy, \n                        \n                        Musical',
    'English',
    'Victor Fleming',
    'Victor Fleming, \n                            \n                        \n                            \n                                Mervyn LeRoy',
    'L. Frank Baum, \n                            \n                        \n                            \n                                Noel Langley, \n                            \n                        \n                            \n                                Florence Ryerson, \n                            \n                        \n                            \n                                Edgar Allan Woolf',
    'Aug 25, 1939\n\xa0wide',
    'Aug 12, 2003',
    '$5.5M',
    '1h 41m',
    'Metro-Goldwyn-Mayer',
    'Mono',
    'Flat (1.37:1)'],
   'detail_crew': ['Judy Garland',
    'Frank Morgan',
    'Ray Bolger',
    'Bert Lahr',
    'Jack Haley',
    'Billie Burke',
    'Margaret Hamilton',
    'Charley Grapewin',
    'Pat Walshe',
    'Clara Blandick',
    'Victor Fleming',
    'L. Frank Baum',
    'Noel Langley',
    'Florence Ryerson',
    'Edgar Allan Woolf',
    'Victor Fleming',
    'Mervyn LeRoy',
    'Harold Arlen',
    'Herbert Stothart',
    'Felix Mendelssohn',
    'Modest Mussorgsky',
    'Nikolai Rimsky-Korsakov',
    'Robert Schumann',
    'Harold Rosson',
    'Blanche Sewell',
    'Cedric Gibbons',
    'Edwin B. Willis',
    'Adrian',
    'Douglas Shearer']},
  'Modern Times (1936)': {'detail_info': ['G',
    'Comedy',
    'English',
    'Charlie Chaplin',
    'Charlie Chaplin',
    'Charlie Chaplin',
    'Feb 5, 1936\n\xa0wide',
    'Nov 16, 2010',
    '$163.2K',
    '1h 29m',
    'United Artists',
    'Mono',
    'Flat (1.37:1)'],
   'detail_crew': ['Charlie Chaplin',
    'Paulette Goddard',
    'Henry Bergman',
    'Tiny Sandford',
    'Chester Conklin',
    'Hank Mann',
    'Stanley Blystone',
    'Cecil Reynolds',
    'Charlie Chaplin',
    'Charlie Chaplin',
    'Charlie Chaplin',
    'Charlie Chaplin',
    'Ira H. Morgan',
    'Roland Totheroh',
    'Alfred Reeves',
    'Jack Wilson',
    'Carter DeHaven']}},
 'Top 100 Comedy Movies': {'It Happened One Night (1934)': {'detail_info': ['Romance, \n                        \n                        Comedy',
    'English',
    'Frank Capra',
    'Frank Capra, \n                            \n                        \n                            \n                                Harry Cohn',
    'Samuel Hopkins Adams, \n                            \n                        \n                            \n                                Robert Riskin',
    'Feb 22, 1934\n\xa0original',
    'Dec 28, 1999',
    '1h 45m',
    'Columbia Pictures, \n                        \n                        Columbia TriStar Home Video, \n                        \n                        RCA/Columbia',
    'Mono',
    '35mm, Flat (1.37:1)'],
   'detail_crew': ['Claudette Colbert',
    'Clark Gable',
    'Walter Connolly',
    'Roscoe Karns',
    'Alan Hale',
    'Ward Bond',
    'Frank Capra',
    'Frank Capra',
    'Howard Jackson',
    'Samuel Hopkins Adams',
    'Robert Riskin',
    'Harry Cohn',
    'Frank Capra',
    'Harry Cohn',
    'Howard Jackson',
    'Louis Silvers',
    'Joseph Walker',
    'Gene Havlick',
    'Stephen Goosson',
    'Robert Kalloch']},
  'Modern Times (1936)': {'detail_info': ['G',
    'Comedy',
    'English',
    'Charlie Chaplin',
    'Charlie Chaplin',
    'Charlie Chaplin',
    'Feb 5, 1936\n\xa0wide',
    'Nov 16, 2010',
    '$163.2K',
    '1h 29m',
    'United Artists',
    'Mono',
    'Flat (1.37:1)'],
   'detail_crew': ['Charlie Chaplin',
    'Paulette Goddard',
    'Henry Bergman',
    'Tiny Sandford',
    'Chester Conklin',
    'Hank Mann',
    'Stanley Blystone',
    'Cecil Reynolds',
    'Charlie Chaplin',
    'Charlie Chaplin',
    'Charlie Chaplin',
    'Charlie Chaplin',
    'Ira H. Morgan',
    'Roland Totheroh',
    'Alfred Reeves',
    'Jack Wilson',
    'Carter DeHaven']},
  'Toy Story 4 (2019)': {'detail_info': ['G',
    'Comedy, \n                        \n                        Kids & family, \n                        \n                        Adventure, \n                        \n                        Fantasy, \n                        \n                        Animation',
    'English',
    'Josh Cooley',
    'Mark Nielsen, \n                            \n                        \n                            \n                                Jonas Rivera, \n                            \n                        \n                            \n                                Galyn Susman',
    'Andrew Stanton, \n                            \n                        \n                            \n                                Stephany Folsom',
    'Jun 21, 2019\n\xa0wide',
    'Oct 1, 2019',
    '$434.0M',
    '1h 40m',
    'Walt Disney',
    'DTS, Dolby Digital, Dolby Atmos',
    'Scope (2.35:1)',
    'Pixar'],
   'detail_crew': ['Tom Hanks',
    'Tim Allen',
    'Annie Potts',
    'Tony Hale',
    'Keegan-Michael Key',
    'Madeleine McGraw',
    'Christina Hendricks',
    'Jordan Peele',
    'Keanu Reeves',
    'Ally Maki',
    'Jay Hernandez',
    'Lori Alan',
    'Joan Cusack',
    'Bonnie Hunt',
    'Kristen Schaal',
    'Emily Davis',
    'Wallace Shawn',
    'John Ratzenberger',
    'Blake Clark',
    'June Squibb',
    'Josh Cooley',
    'Andrew Stanton',
    'Stephany Folsom',
    'Mark Nielsen',
    'Jonas Rivera',
    'Galyn Susman',
    'Andrew Stanton',
    'Lee Unkrich',
    'Pete Docter',
    'Patrick Lin',
    'Jean-Claude Kalache',
    'Axel Geddes',
    'Randy Newman',
    'Bob Pauley',
    'Kevin Reher',
    'Natalie Lyon']},
  'Lady Bird (2017)': {'detail_info': ['R (Language|Brief Graphic Nudity|Sexual Content|Teen Partying)',
    'Drama, \n                        \n                        Comedy',
    'English',
    'Greta Gerwig',
    "Scott Rudin, \n                            \n                        \n                            \n                                Eli Bush, \n                            \n                        \n                            \n                                Evelyn O'Neill",
    'Greta Gerwig',
    'Nov 22, 2017\n\xa0wide',
    'Mar 6, 2018',
    '$48.9M',
    '1h 34m',
    'A24',
    'Dolby Digital',
    'Flat (1.85:1)'],
   'detail_crew': ['Saoirse Ronan',
    'Laurie Metcalf',
    'Timothée Chalamet',
    'Lucas Hedges',
    'Tracy Letts',
    'Odeya Rush',
    'Jake McDorman',
    'Beanie Feldstein',
    'Lois Smith',
    'Stephen Henderson',
    'Jordan Rodrigues',
    'Marielle Scott',
    'John Karna',
    'Bayne Gibby',
    'Laura Marano',
    'Greta Gerwig',
    'Greta Gerwig',
    'Scott Rudin',
    'Eli Bush',
    "Evelyn O'Neill",
    'Lila Yacoub',
    'Sam Levy',
    'Nick Houy',
    'Jon Brion',
    'Chris Jones',
    'April Napier',
    'Allison Jones',
    'Heidi Griffiths',
    'Jordan Thaler']}},
 'Top 100 Documentary Movies': {"Won't You Be My Neighbor? (2018)": {'detail_info': ['PG-13 (Some Thematic Elements|Language)',
    'Documentary',
    'English',
    'Morgan Neville',
    'Caryn Capotosto, \n                            \n                        \n                            \n                                Nicholas Ma',
    'Jun 29, 2018\n\xa0wide',
    'Sep 4, 2018',
    '$22.6M',
    '1h 34m',
    'Focus Features'],
   'detail_crew': ['Joanne Rogers',
    'Bill Isler',
    'Hedda Sharapan',
    'Margaret Whitmer',
    'Tom Junod',
    'Betty Seamans',
    'Morgan Neville',
    'Leslie Berriman',
    'John Boccardo',
    'Caryn Capotosto',
    'Nicholas Ma',
    'Graham Willoughby',
    'Jeff Malmberg',
    'Aaron Wickenden',
    'Jonathan Kirkscey']},
  'I Am Not Your Negro (2017)': {'detail_info': ['PG-13 (Disturbing Violent Images|Brief Nudity|Language|Thematic Material)',
    'Documentary',
    'English',
    'Raoul Peck',
    'Rémi Grellety, \n                            \n                        \n                            \n                                Raoul Peck, \n                            \n                        \n                            \n                                Hébert Peck',
    'James Baldwin',
    'Feb 3, 2017\n\xa0limited',
    'May 2, 2017',
    '$7.1M',
    '1h 35m',
    'Magnolia Pictures, \n                        \n                        Amazon Studios'],
   'detail_crew': ['Samuel L. Jackson',
    'Raoul Peck',
    'James Baldwin',
    'Rémi Grellety',
    'Raoul Peck',
    'Hébert Peck',
    'Henry Adebonojo',
    'Bill Ross IV',
    'Turner Ross',
    'Alexandra Strauss']},
  'Apollo 11 (2019)': {'detail_info': ['G',
    'Drama, \n                        \n                        History, \n                        \n                        Documentary',
    'English',
    'Todd Douglas Miller',
    'Todd Douglas Miller, \n                            \n                        \n                            \n                                Thomas Petersen, \n                            \n                        \n                            \n                                Evan Krauss',
    'Mar 8, 2019\n\xa0wide',
    'May 14, 2019',
    '$9.0M',
    '1h 33m',
    'CNN Films, \n                        \n                        Neon',
    'Dolby Digital'],
   'detail_crew': ['Bill Anders',
    'David Reed',
    'Todd Douglas Miller',
    'Courtney Sexton',
    'Amy Entelis',
    'Josh Braun',
    'Todd Douglas Miller',
    'Thomas Petersen',
    'Evan Krauss',
    'Edwin "Buzz" Aldrin',
    'Todd Douglas Miller',
    'Matt Morton']},
  'Summer of Soul (...Or, When the Revolution Could Not Be Televised) (2021)': {'detail_info': ['PG-13 (Smoking|Brief Drug Material|Some Disturbing Images)',
    'Music, \n                        \n                        Documentary',
    'English',
    'Ahmir "Questlove" Thompson',
    'David Dinerstein, \n                            \n                        \n                            \n                                Robert Fyvolent, \n                            \n                        \n                            \n                                Joseph Patel',
    'Jul 2, 2021\n\xa0wide',
    'Jul 2, 2021',
    '$2.3M',
    '1h 57m',
    'Searchlight Pictures',
    'Dolby Digital, Stereo'],
   'detail_crew': ['Lin-Manuel Miranda',
    'Stevie Wonder',
    'B.B. King',
    'Chris Rock',
    'Nina Simone',
    'Stevie Wonder',
    'Nina Simone',
    'Sly and the Family Stone',
    'Gladys Knight and the Pips',
    'Mahalia Jackson',
    'B.B. King',
    'The 5th Dimension',
    'Ahmir "Questlove" Thompson',
    'David Dinerstein',
    'Robert Fyvolent',
    'Joseph Patel',
    'Jody Allen',
    'Rocky Collins',
    'Shawn Peters',
    'Joshua L. Pearson',
    'Jen Isaacson',
    'Jon Kamen',
    'David Sirulnick',
    'Jannat Gargi',
    'Beth Hubbard',
    'Davis Guggenheim',
    'Laurene Powell-Jobs',
    'Jeffrey Lurie',
    'Marie Therese Guirgis']}},
 'Top 100 Drama Movies': {'Citizen Kane (1941)': {'detail_info': ['PG',
    'Drama',
    'English',
    'Orson Welles',
    'Orson Welles',
    'Herman J. Mankiewicz, \n                            \n                        \n                            \n                                Orson Welles, \n                            \n                        \n                            \n                                John Houseman',
    'May 1, 1941\n\xa0wide',
    'Feb 23, 2010',
    '$977.3K',
    '1h 59m',
    'RKO Radio Pictures',
    'Mono',
    'Academy (1.33:1)'],
   'detail_crew': ['Orson Welles',
    'Joseph Cotten',
    'Ruth Warrick',
    'Everett Sloane',
    'Agnes Moorehead',
    'Dorothy Comingore',
    'Ray Collins',
    'George Coulouris',
    'William Alland',
    'Paul Stewart',
    'Erskine Sanford',
    'Orson Welles',
    'Herman J. Mankiewicz',
    'Orson Welles',
    'John Houseman',
    'Orson Welles',
    'Bernard Herrmann',
    'Gregg Toland',
    'Robert Wise',
    'Van Nest Polglase',
    'Perry Ferguson',
    'Darrell Silvera',
    'Edward Stevenson',
    'Maurice Seiderman',
    'John Aalberg',
    'Russell A. Cully',
    'Vernon L. Walker']},
  'Black Panther (2018)': {'detail_info': ['PG-13 (Sequences of Action Violence|A Brief Rude Gesture)',
    'Adventure, \n                        \n                        Fantasy, \n                        \n                        Action',
    'English',
    'Ryan Coogler',
    'Kevin Feige',
    'Ryan Coogler, \n                            \n                        \n                            \n                                Joe Robert Cole',
    'Feb 16, 2018\n\xa0wide',
    'May 2, 2018',
    '$700.2M',
    '2h 14m',
    'Walt Disney',
    'DTS, Dolby Atmos',
    'Scope (2.35:1)',
    'Marvel Cinematic Universe'],
   'detail_crew': ['Chadwick Boseman',
    'Michael B. Jordan',
    "Lupita Nyong'o",
    'Danai Gurira',
    'Martin Freeman',
    'Daniel Kaluuya',
    'Letitia Wright',
    'Winston Duke',
    'Angela Bassett',
    'Forest Whitaker',
    'Andy Serkis',
    'Florence Kasumba',
    'Sterling K. Brown',
    'John Kani',
    'Ryan Coogler',
    'Ryan Coogler',
    'Joe Robert Cole',
    'Kevin Feige',
    "Louis D'Esposito",
    'Victoria Alonso',
    'Nate Moore',
    'Jeffrey Chernov',
    'Stan Lee',
    'Rachel Morrison',
    'Debbie Berman',
    'Michael P. Shawver',
    'Ludwig Göransson',
    'Hannah Beachler',
    'Ruth E. Carter',
    'Sarah Halley Finn']},
  'Parasite (Gisaengchung) (2019)': {'detail_info': ['R (Sexual Content|Language|Some Violence)',
    'Drama, \n                        \n                        Mystery & thriller, \n                        \n                        Comedy',
    'Korean',
    'Bong Joon Ho',
    'Kwak Sin-ae, \n                            \n                        \n                            \n                                Moon Yanggwon',
    'Bong Joon Ho, \n                            \n                        \n                            \n                                Han Jinwon',
    'Nov 1, 2019\n\xa0wide',
    'Oct 11, 2019',
    '$53.4M',
    '2h 12m',
    'Neon',
    'Dolby Atmos, Dolby Digital',
    'Scope (2.35:1)'],
   'detail_crew': ['Song Kang-ho',
    'Jo Yeo-jeong',
    'Park So-dam',
    'Choi Woo-sik',
    'Lee Sun-gyun',
    'Park Seo-joon',
    'Jung Ji-so',
    'Lee Jeong-eun',
    'Andreas Fronk',
    'Jang Hye-jin',
    'Park Myeong-hoon',
    'Lee Ji-hye',
    'Jung Hyun-jun',
    'Park Keun-rok',
    'Lee Joo-hyung',
    'Lee Dong-yong',
    'Bong Joon Ho',
    'Bong Joon Ho',
    'Han Jinwon',
    'Miky Lee',
    'Kwak Sin-ae',
    'Moon Yanggwon',
    'Hong Kyung-pyo',
    'Kim Chang-ho',
    'Yang Jin-mo',
    'Jung Jae-il',
    'Lee Ha-jun',
    'Choi Seyeon']},
  'Avengers: Endgame (2019)': {'detail_info': ['PG-13 (Sequences of Sci-Fi Violence|Action|Some Language)',
    'Sci-fi, \n                        \n                        Adventure, \n                        \n                        Fantasy, \n                        \n                        Action',
    'English',
    'Anthony Russo, \n                            \n                        \n                            \n                                Joe Russo',
    'Kevin Feige',
    'Christopher Markus, \n                            \n                        \n                            \n                                Stephen McFeely',
    'Apr 26, 2019\n\xa0wide',
    'Jul 30, 2019',
    '$858.4M',
    '3h 1m',
    'Walt Disney',
    'Dolby Atmos, DTS, Dolby Digital, SDDS',
    'Scope (2.35:1)',
    'Marvel Cinematic Universe'],
   'detail_crew': ['Robert Downey Jr.',
    'Chris Evans',
    'Mark Ruffalo',
    'Chris Hemsworth',
    'Scarlett Johansson',
    'Jeremy Renner',
    'Brie Larson',
    'Paul Rudd',
    'Don Cheadle',
    'Karen Gillan',
    'Danai Gurira',
    'Bradley Cooper',
    'Gwyneth Paltrow',
    'Jon Favreau',
    'Benedict Wong',
    'Tessa Thompson',
    'Josh Brolin',
    'Tilda Swinton',
    'Robert Redford',
    'Dave Bautista',
    'Anthony Russo',
    'Joe Russo',
    'Christopher Markus',
    'Stephen McFeely',
    'Kevin Feige',
    "Louis D'Esposito",
    'Victoria Alonso',
    'Michael Grillo',
    'Trinh Tran',
    'Jon Favreau',
    'James Gunn',
    'Stan Lee',
    'Trent Opaloch',
    'Jeffrey Ford',
    'Matthew Schmidt',
    'Alan Silvestri',
    'Charles Wood',
    'Judianna Makovsky',
    'Sarah Halley Finn']}},
 'Top 100 Horror Movies': {'Us (2019)': {'detail_info': ['R (Violence/Terror|Language)',
    'Horror, \n                        \n                        Mystery & thriller',
    'English',
    'Jordan Peele',
    'Jordan Peele, \n                            \n                        \n                            \n                                Sean McKittrick, \n                            \n                        \n                            \n                                Jason Blum, \n                            \n                        \n                            \n                                Ian Cooper',
    'Jordan Peele',
    'Mar 22, 2019\n\xa0wide',
    'Jun 4, 2019',
    '$175.0M',
    '1h 56m',
    'Universal Pictures',
    'Dolby Atmos, DTS',
    'Scope (2.35:1)'],
   'detail_crew': ["Lupita Nyong'o",
    'Winston Duke',
    'Elisabeth Moss',
    'Tim Heidecker',
    'Yahya Abdul-Mateen II',
    'Anna Diop',
    'Evan Alex',
    'Shahadi Wright Joseph',
    'Madison Curry',
    'Cali Sheldon',
    'Noelle Sheldon',
    'Duke Nicholson',
    'Jordan Peele',
    'Jordan Peele',
    'Jordan Peele',
    'Sean McKittrick',
    'Jason Blum',
    'Ian Cooper',
    'Daniel Lupi',
    'Beatriz Sequeira',
    'Mike Gioulakis',
    'Nicholas Monsour',
    'Michael Abels',
    'Ruth De Jong',
    'Kym Barrett',
    'Terri Taylor']},
  'Get Out (2017)': {'detail_info': ['R (Language|Bloody Images|Sexual References|Violence)',
    'Horror, \n                        \n                        Mystery & thriller, \n                        \n                        Comedy',
    'English',
    'Jordan Peele',
    'Sean McKittrick, \n                            \n                        \n                            \n                                Jason Blum, \n                            \n                        \n                            \n                                Ted Hamm, \n                            \n                        \n                            \n                                Jordan Peele',
    'Jordan Peele',
    'Feb 24, 2017\n\xa0wide',
    'May 23, 2017',
    '$176.0M',
    '1h 44m',
    'Universal Pictures',
    'Dolby Digital, DTS, SDDS',
    'Scope (2.35:1)'],
   'detail_crew': ['Daniel Kaluuya',
    'Allison Williams',
    'Catherine Keener',
    'Bradley Whitford',
    'Caleb Landry Jones',
    'Marcus Henderson',
    'Betty Gabriel',
    'LaKeith Stanfield',
    'Stephen Root',
    'Lil Rel Howery',
    'Ashley LeConte Campbell',
    'John Wilmot',
    'Caren L. Larkey',
    'Julie Ann Doan',
    'Rutherford Cravens',
    'Geraldine Singer',
    'Jordan Peele',
    'Jordan Peele',
    'Raymond Mansfield',
    'Couper Samuelson',
    'Shaun Redick',
    'Jeanette Volturno',
    'Sean McKittrick',
    'Jason Blum',
    'Ted Hamm',
    'Jordan Peele',
    'Toby Oliver',
    'Gregory Plotkin',
    'Michael Abels',
    'Rusty Smith',
    'Chris Craine',
    'Leonard R. Spears',
    'Nadine Haders',
    'Terri Taylor']},
  'The Cabinet of Dr. Caligari (Das Cabinet des Dr. Caligari) (1920)': {'detail_info': ['Horror, \n                        \n                        Fantasy, \n                        \n                        Mystery & thriller',
    'German',
    'Robert Wiene',
    'Rudolf Meinert, \n                            \n                        \n                            \n                                Erich Pommer',
    'Hans Janowitz, \n                            \n                        \n                            \n                                Carl Mayer',
    'Mar 19, 1921\n\xa0wide',
    'Feb 16, 1999',
    '1h 9m',
    'Grapevine Video, \n                        \n                        Kino Video'],
   'detail_crew': ['Werner Krauss',
    'Conrad Veidt',
    'Lil Dagover',
    'Friedrich Feher',
    'Rudolf Lettinger',
    'Hans Heinrich von Twardowski',
    'Rudolf Klein-Rogge',
    'Robert Wiene',
    'Hans Janowitz',
    'Carl Mayer',
    'Rudolf Meinert',
    'Erich Pommer',
    'Albert Band',
    'Giuseppe Becce',
    'Willy Hameister']},
  'A Quiet Place (2018)': {'detail_info': ['PG-13 (Terror and Some Bloody Images)',
    'Horror, \n                        \n                        Mystery & thriller',
    'English',
    'John Krasinski',
    'Michael Bay, \n                            \n                        \n                            \n                                Andrew Form, \n                            \n                        \n                            \n                                Brad Fuller',
    'Bryan Woods, \n                            \n                        \n                            \n                                Scott Beck, \n                            \n                        \n                            \n                                John Krasinski',
    'Apr 6, 2018\n\xa0wide',
    'Jul 10, 2018',
    '$68.4M',
    '1h 30m',
    'Paramount Pictures',
    'Dolby Atmos, Dolby Digital, DTS, SDDS',
    'Scope (2.35:1)'],
   'detail_crew': ['Emily Blunt',
    'John Krasinski',
    'Millicent Simmonds',
    'Noah Jupe',
    'Cade Woodward',
    'Leon Russom',
    'Rhoda Pell',
    'Evangelina Cavoli',
    'John Krasinski',
    'Bryan Woods',
    'Scott Beck',
    'John Krasinski',
    'Michael Bay',
    'Andrew Form',
    'Brad Fuller',
    'John Krasinski',
    'Celia D. Costas',
    'Allyson Seeger',
    'Scott Beck',
    'Bryan Woods',
    'Aaron Janus',
    'Charlotte Bruus Christensen',
    'Christopher Tellefsen',
    'Marco Beltrami',
    'Jeffrey Beecroft',
    'Kasia Walicka-Maimone']}},
 'Top 100 Kids & Family Movies': {'The Wizard of Oz (1939)': {'detail_info': ['G',
    'Fantasy, \n                        \n                        Musical',
    'English',
    'Victor Fleming',
    'Victor Fleming, \n                            \n                        \n                            \n                                Mervyn LeRoy',
    'L. Frank Baum, \n                            \n                        \n                            \n                                Noel Langley, \n                            \n                        \n                            \n                                Florence Ryerson, \n                            \n                        \n                            \n                                Edgar Allan Woolf',
    'Aug 25, 1939\n\xa0wide',
    'Aug 12, 2003',
    '$5.5M',
    '1h 41m',
    'Metro-Goldwyn-Mayer',
    'Mono',
    'Flat (1.37:1)'],
   'detail_crew': ['Judy Garland',
    'Frank Morgan',
    'Ray Bolger',
    'Bert Lahr',
    'Jack Haley',
    'Billie Burke',
    'Margaret Hamilton',
    'Charley Grapewin',
    'Pat Walshe',
    'Clara Blandick',
    'Victor Fleming',
    'L. Frank Baum',
    'Noel Langley',
    'Florence Ryerson',
    'Edgar Allan Woolf',
    'Victor Fleming',
    'Mervyn LeRoy',
    'Harold Arlen',
    'Herbert Stothart',
    'Felix Mendelssohn',
    'Modest Mussorgsky',
    'Nikolai Rimsky-Korsakov',
    'Robert Schumann',
    'Harold Rosson',
    'Blanche Sewell',
    'Cedric Gibbons',
    'Edwin B. Willis',
    'Adrian',
    'Douglas Shearer']},
  'Toy Story 4 (2019)': {'detail_info': ['G',
    'Comedy, \n                        \n                        Kids & family, \n                        \n                        Adventure, \n                        \n                        Fantasy, \n                        \n                        Animation',
    'English',
    'Josh Cooley',
    'Mark Nielsen, \n                            \n                        \n                            \n                                Jonas Rivera, \n                            \n                        \n                            \n                                Galyn Susman',
    'Andrew Stanton, \n                            \n                        \n                            \n                                Stephany Folsom',
    'Jun 21, 2019\n\xa0wide',
    'Oct 1, 2019',
    '$434.0M',
    '1h 40m',
    'Walt Disney',
    'DTS, Dolby Digital, Dolby Atmos',
    'Scope (2.35:1)',
    'Pixar'],
   'detail_crew': ['Tom Hanks',
    'Tim Allen',
    'Annie Potts',
    'Tony Hale',
    'Keegan-Michael Key',
    'Madeleine McGraw',
    'Christina Hendricks',
    'Jordan Peele',
    'Keanu Reeves',
    'Ally Maki',
    'Jay Hernandez',
    'Lori Alan',
    'Joan Cusack',
    'Bonnie Hunt',
    'Kristen Schaal',
    'Emily Davis',
    'Wallace Shawn',
    'John Ratzenberger',
    'Blake Clark',
    'June Squibb',
    'Josh Cooley',
    'Andrew Stanton',
    'Stephany Folsom',
    'Mark Nielsen',
    'Jonas Rivera',
    'Galyn Susman',
    'Andrew Stanton',
    'Lee Unkrich',
    'Pete Docter',
    'Patrick Lin',
    'Jean-Claude Kalache',
    'Axel Geddes',
    'Randy Newman',
    'Bob Pauley',
    'Kevin Reher',
    'Natalie Lyon']},
  'Spider-Man: Into the Spider-Verse (2018)': {'detail_info': ['PG (Mild Language|Frenetic Action Violence|Thematic Elements)',
    'Comedy, \n                        \n                        Kids & family, \n                        \n                        Adventure, \n                        \n                        Fantasy, \n                        \n                        Animation, \n                        \n                        Action',
    'English',
    'Bob Persichetti, \n                            \n                        \n                            \n                                Peter Ramsey, \n                            \n                        \n                            \n                                Rodney Rothman',
    'Avi Arad, \n                            \n                        \n                            \n                                Amy Pascal, \n                            \n                        \n                            \n                                Phil Lord, \n                            \n                        \n                            \n                                Chris Miller, \n                            \n                        \n                            \n                                Christina Steinberg',
    'Phil Lord, \n                            \n                        \n                            \n                                Rodney Rothman',
    'Dec 14, 2018\n\xa0wide',
    'Mar 19, 2019',
    '$190.2M',
    '1h 57m',
    'Sony Pictures Entertainment, \n                        \n                        Columbia Pictures',
    'Dolby Atmos, DTS, Dolby Digital, SDDS',
    'Scope (2.35:1)'],
   'detail_crew': ['Shameik Moore',
    'Jake Johnson',
    'Hailee Steinfeld',
    'Mahershala Ali',
    'Brian Tyree Henry',
    'Lily Tomlin',
    'Luna Lauren Velez',
    'Zoë Kravitz',
    'John Mulaney',
    'Kimiko Glenn',
    'Nicolas Cage',
    'Kathryn Hahn',
    'Liev Schreiber',
    'Chris Pine',
    'Natalie Morales',
    'Bob Persichetti',
    'Peter Ramsey',
    'Rodney Rothman',
    'Phil Lord',
    'Rodney Rothman',
    'Stan Lee',
    'Brian Michael Bendis',
    'Will Allegra',
    'Avi Arad',
    'Amy Pascal',
    'Phil Lord',
    'Chris Miller',
    'Christina Steinberg',
    'Robert Fisher Jr.',
    'Daniel Pemberton',
    'Justin K. Thompson',
    'Dean Gordon',
    "Patrick O'Keefe",
    'Mary Hidalgo']},
  'Inside Out (2015)': {'detail_info': ['PG (Some Action|Mild Thematic Elements)',
    'Animation, \n                        \n                        Kids & family, \n                        \n                        Fantasy, \n                        \n                        Comedy',
    'English',
    'Pete Docter',
    'Jonas Rivera',
    'Pete Docter, \n                            \n                        \n                            \n                                Meg LeFauve, \n                            \n                        \n                            \n                                Josh Cooley',
    'Jun 19, 2015\n\xa0wide',
    'Nov 3, 2015',
    '$2.0M',
    '1h 35m',
    'Walt Disney',
    'Dolby Atmos, Datasat',
    'Pixar'],
   'detail_crew': ['Amy Poehler',
    'Phyllis Smith',
    'Richard Kind',
    'Bill Hader',
    'Lewis Black',
    'Mindy Kaling',
    'Kaitlyn Dias',
    'Diane Lane',
    'Kyle MacLachlan',
    'Pete Docter',
    'Ronnie del Carmen',
    'Pete Docter',
    'Meg LeFauve',
    'Josh Cooley',
    'Jonas Rivera',
    'Patrick Lin',
    'Kevin Nolting',
    'Michael Giacchino',
    'Ralph Eggleston',
    'Kevin Reher',
    'Natalie Lyon']}},
 'Top 100 Musical & Performing Arts Movies': {'The Wizard of Oz (1939)': {'detail_info': ['G',
    'Fantasy, \n                        \n                        Musical',
    'English',
    'Victor Fleming',
    'Victor Fleming, \n                            \n                        \n                            \n                                Mervyn LeRoy',
    'L. Frank Baum, \n                            \n                        \n                            \n                                Noel Langley, \n                            \n                        \n                            \n                                Florence Ryerson, \n                            \n                        \n                            \n                                Edgar Allan Woolf',
    'Aug 25, 1939\n\xa0wide',
    'Aug 12, 2003',
    '$5.5M',
    '1h 41m',
    'Metro-Goldwyn-Mayer',
    'Mono',
    'Flat (1.37:1)'],
   'detail_crew': ['Judy Garland',
    'Frank Morgan',
    'Ray Bolger',
    'Bert Lahr',
    'Jack Haley',
    'Billie Burke',
    'Margaret Hamilton',
    'Charley Grapewin',
    'Pat Walshe',
    'Clara Blandick',
    'Victor Fleming',
    'L. Frank Baum',
    'Noel Langley',
    'Florence Ryerson',
    'Edgar Allan Woolf',
    'Victor Fleming',
    'Mervyn LeRoy',
    'Harold Arlen',
    'Herbert Stothart',
    'Felix Mendelssohn',
    'Modest Mussorgsky',
    'Nikolai Rimsky-Korsakov',
    'Robert Schumann',
    'Harold Rosson',
    'Blanche Sewell',
    'Cedric Gibbons',
    'Edwin B. Willis',
    'Adrian',
    'Douglas Shearer']},
  'A Night at the Opera (1935)': {'detail_info': ['Comedy',
    'English',
    'Sam Wood',
    'Nov 15, 1935\n\xa0original',
    'May 4, 2004',
    '1h 33m',
    'Metro-Goldwyn-Mayer'],
   'detail_crew': ['Groucho Marx',
    'Harpo Marx',
    'Kitty Carlisle Hart',
    'Chico Marx',
    'Margaret Dumont',
    'Allan Jones',
    'Walter Woolf King',
    'Sig Ruman',
    'Sam Wood']},
  "Singin' in the Rain (1952)": {'detail_info': ['G',
    'Comedy, \n                        \n                        Musical',
    'English',
    'Stanley Donen, \n                            \n                        \n                            \n                                Gene Kelly',
    'Arthur Freed',
    'Betty Comden, \n                            \n                        \n                            \n                                Adolph Green',
    'Apr 10, 1952\n\xa0wide',
    'Sep 24, 2002',
    '$23.9K',
    '1h 42m',
    'Metro-Goldwyn-Mayer',
    'Mono',
    'Flat (1.37:1)'],
   'detail_crew': ['Gene Kelly',
    'Debbie Reynolds',
    "Donald O'Connor",
    'Jean Hagen',
    'Millard Mitchell',
    'Rita Moreno',
    'Douglas V. Fowley',
    'Cyd Charisse',
    'Madge Blake',
    'King Donovan',
    'Kathleen Freeman',
    'Bobby Watson',
    'Jimmy Thompson',
    'Dan Foster',
    'Margaret Bert',
    'Mae Clarke',
    'John Dodsworth',
    'Stuart Holmes',
    'Stanley Donen',
    'Gene Kelly',
    'Betty Comden',
    'Adolph Green',
    'Arthur Freed',
    'Nacio Herb Brown',
    'Lennie Hayton',
    'Harold Rosson',
    'Adrienne Fazan',
    'Randall Duell',
    'Cedric Gibbons',
    'Jacques Mapes',
    'Edwin B. Willis',
    'Walter Plunkett',
    'Sydney Guilaroff',
    'William Tuttle',
    'Douglas Shearer',
    'Warren Newcombe',
    'Irving G. Ries',
    'Lennie Hayton']},
  'An American in Paris (1951)': {'detail_info': ['Romance, \n                        \n                        Musical',
    'English',
    'Vincente Minnelli',
    'Arthur Freed',
    'Alan Jay Lerner',
    'Nov 9, 1951\n\xa0wide',
    'Dec 2, 2004',
    '1h 53m',
    'Metro-Goldwyn-Mayer',
    'Mono',
    'Flat (1.37:1)'],
   'detail_crew': ['Gene Kelly',
    'Leslie Caron',
    'Oscar Levant',
    'Georges Guétary',
    'Nina Foch',
    'Eugene Borden',
    'Martha Barnattre',
    'Ann Codee',
    'George Davis',
    'Hayden Rorke',
    'Paul Maxey',
    'Richard Wessel',
    'Vincente Minnelli',
    'Alan Jay Lerner',
    'Roger Edens',
    'Arthur Freed',
    'Saul Chaplin',
    'George Gershwin',
    'John Alton',
    'Alfred Gilks',
    'Adrienne Fazan',
    'E. Preston Ames',
    'Cedric Gibbons',
    'F. Keogh Gleason',
    'Edwin B. Willis',
    'Orry-Kelly',
    'Walter Plunkett',
    'Irene Sharaff',
    'Sydney Guilaroff',
    'William Tuttle',
    'Robert J. Schiffer',
    'Johnny Green']}},
 'Top 100 Mystery & Suspense Movies': {'Citizen Kane (1941)': {'detail_info': ['PG',
    'Drama',
    'English',
    'Orson Welles',
    'Orson Welles',
    'Herman J. Mankiewicz, \n                            \n                        \n                            \n                                Orson Welles, \n                            \n                        \n                            \n                                John Houseman',
    'May 1, 1941\n\xa0wide',
    'Feb 23, 2010',
    '$977.3K',
    '1h 59m',
    'RKO Radio Pictures',
    'Mono',
    'Academy (1.33:1)'],
   'detail_crew': ['Orson Welles',
    'Joseph Cotten',
    'Ruth Warrick',
    'Everett Sloane',
    'Agnes Moorehead',
    'Dorothy Comingore',
    'Ray Collins',
    'George Coulouris',
    'William Alland',
    'Paul Stewart',
    'Erskine Sanford',
    'Orson Welles',
    'Herman J. Mankiewicz',
    'Orson Welles',
    'John Houseman',
    'Orson Welles',
    'Bernard Herrmann',
    'Gregg Toland',
    'Robert Wise',
    'Van Nest Polglase',
    'Perry Ferguson',
    'Darrell Silvera',
    'Edward Stevenson',
    'Maurice Seiderman',
    'John Aalberg',
    'Russell A. Cully',
    'Vernon L. Walker']},
  'Knives Out (2019)': {'detail_info': ['PG-13 (Drug Material|Brief Violence|Sexual References|Some Strong Language|Thematic Elements)',
    'Drama, \n                        \n                        Mystery & thriller, \n                        \n                        Crime, \n                        \n                        Comedy',
    'English',
    'Rian Johnson',
    'Rian Johnson, \n                            \n                        \n                            \n                                Ram Bergman, \n                            \n                        \n                            \n                                Jonathan Golfman, \n                            \n                        \n                            \n                                Brye Adler',
    'Rian Johnson',
    'Nov 27, 2019\n\xa0wide',
    'Jun 12, 2020',
    '$165.4M',
    '2h 10m',
    'Lionsgate Films',
    'Dolby',
    'Flat (1.85:1)'],
   'detail_crew': ['Daniel Craig',
    'Chris Evans',
    'Ana de Armas',
    'Jamie Lee Curtis',
    'Toni Collette',
    'Don Johnson',
    'Michael Shannon',
    'LaKeith Stanfield',
    'Katherine Langford',
    'Jaeden Martell',
    'Noah Segan',
    'Edi Patterson',
    'Riki Lindhome',
    'Christopher Plummer',
    'Frank Oz',
    'Raúl Castillo',
    'K Callan',
    'Rian Johnson',
    'Rian Johnson',
    'Rian Johnson',
    'Ram Bergman',
    'Jonathan Golfman',
    'Brye Adler',
    'Tom Karnowski',
    'Nathan Johnson',
    'Steve Yedlin',
    'Bob Ducsay',
    'David Crank',
    'Jeremy Woodward',
    'David Schlesinger',
    'Jenny Eagan',
    'Mary Vernieu']},
  'Us (2019)': {'detail_info': ['R (Violence/Terror|Language)',
    'Horror, \n                        \n                        Mystery & thriller',
    'English',
    'Jordan Peele',
    'Jordan Peele, \n                            \n                        \n                            \n                                Sean McKittrick, \n                            \n                        \n                            \n                                Jason Blum, \n                            \n                        \n                            \n                                Ian Cooper',
    'Jordan Peele',
    'Mar 22, 2019\n\xa0wide',
    'Jun 4, 2019',
    '$175.0M',
    '1h 56m',
    'Universal Pictures',
    'Dolby Atmos, DTS',
    'Scope (2.35:1)'],
   'detail_crew': ["Lupita Nyong'o",
    'Winston Duke',
    'Elisabeth Moss',
    'Tim Heidecker',
    'Yahya Abdul-Mateen II',
    'Anna Diop',
    'Evan Alex',
    'Shahadi Wright Joseph',
    'Madison Curry',
    'Cali Sheldon',
    'Noelle Sheldon',
    'Duke Nicholson',
    'Jordan Peele',
    'Jordan Peele',
    'Jordan Peele',
    'Sean McKittrick',
    'Jason Blum',
    'Ian Cooper',
    'Daniel Lupi',
    'Beatriz Sequeira',
    'Mike Gioulakis',
    'Nicholas Monsour',
    'Michael Abels',
    'Ruth De Jong',
    'Kym Barrett',
    'Terri Taylor']},
  'Mission: Impossible - Fallout (2018)': {'detail_info': ['PG-13 (Intense Sequences of Action|Brief Strong Language|Violence)',
    'Mystery & thriller, \n                        \n                        Adventure, \n                        \n                        Action',
    'English',
    'Christopher McQuarrie',
    'Tom Cruise, \n                            \n                        \n                            \n                                Christopher McQuarrie, \n                            \n                        \n                            \n                                Jake Myers, \n                            \n                        \n                            \n                                J.J. Abrams',
    'Christopher McQuarrie',
    'Jul 27, 2018\n\xa0wide',
    'Oct 23, 2018',
    '$220.1M',
    '2h 27m',
    'Paramount Pictures',
    'DTS, Dolby Atmos, Dolby Digital',
    'Scope (2.35:1)'],
   'detail_crew': ['Tom Cruise',
    'Henry Cavill',
    'Ving Rhames',
    'Simon Pegg',
    'Rebecca Ferguson',
    'Sean Harris',
    'Angela Bassett',
    'Michelle Monaghan',
    'Alec Baldwin',
    'Vanessa Kirby',
    'Wes Bentley',
    'Frederick Schmidt',
    'Wolf Blitzer',
    'Christopher McQuarrie',
    'Christopher McQuarrie',
    'David Ellison',
    'Dana Goldberg',
    'Don Granger',
    'Tom Cruise',
    'Christopher McQuarrie',
    'Jake Myers',
    'J.J. Abrams',
    'Rob Hardy',
    'Eddie Hamilton',
    'Lorne Balfe',
    'Peter Wenham',
    'Jeffrey Kurland']}},
 'Top 100 Romance Movies': {'It Happened One Night (1934)': {'detail_info': ['Romance, \n                        \n                        Comedy',
    'English',
    'Frank Capra',
    'Frank Capra, \n                            \n                        \n                            \n                                Harry Cohn',
    'Samuel Hopkins Adams, \n                            \n                        \n                            \n                                Robert Riskin',
    'Feb 22, 1934\n\xa0original',
    'Dec 28, 1999',
    '1h 45m',
    'Columbia Pictures, \n                        \n                        Columbia TriStar Home Video, \n                        \n                        RCA/Columbia',
    'Mono',
    '35mm, Flat (1.37:1)'],
   'detail_crew': ['Claudette Colbert',
    'Clark Gable',
    'Walter Connolly',
    'Roscoe Karns',
    'Alan Hale',
    'Ward Bond',
    'Frank Capra',
    'Frank Capra',
    'Howard Jackson',
    'Samuel Hopkins Adams',
    'Robert Riskin',
    'Harry Cohn',
    'Frank Capra',
    'Harry Cohn',
    'Howard Jackson',
    'Louis Silvers',
    'Joseph Walker',
    'Gene Havlick',
    'Stephen Goosson',
    'Robert Kalloch']},
  'Casablanca (1942)': {'detail_info': ['PG',
    'Drama',
    'English',
    'Michael Curtiz',
    'Hal B. Wallis',
    'Murray Burnett, \n                            \n                        \n                            \n                                Joan Alison, \n                            \n                        \n                            \n                                Julius J. Epstein, \n                            \n                        \n                            \n                                Philip G. Epstein, \n                            \n                        \n                            \n                                Howard Koch',
    'Jan 23, 1943\n\xa0wide',
    'Nov 17, 1998',
    '1h 42m',
    'Warner Bros. Pictures',
    'Mono',
    'Flat (1.37:1)'],
   'detail_crew': ['Humphrey Bogart',
    'Ingrid Bergman',
    'Paul Henreid',
    'Claude Rains',
    'Conrad Veidt',
    'Peter Lorre',
    'Sydney Greenstreet',
    'Dooley Wilson',
    'Marcel Dalio',
    'S.Z. Sakall',
    'Joy Page',
    'Michael Curtiz',
    'Murray Burnett',
    'Joan Alison',
    'Julius J. Epstein',
    'Philip G. Epstein',
    'Howard Koch',
    'Hal B. Wallis',
    'Jack L. Warner',
    'M.K. Jerome',
    'Jack Scholl',
    'Max Steiner',
    'Arthur Edeson',
    'Owen Marks',
    'Carl Jules Weyl',
    'George James Hopkins',
    'Perc Westmore',
    'Francis J. Scheid',
    'Lawrence W. Butler',
    'Willard Van Enger']},
  'The Philadelphia Story (1940)': {'detail_info': ['Romance, \n                        \n                        Comedy',
    'English',
    'George Cukor',
    'Joseph L. Mankiewicz',
    'Philip Barry, \n                            \n                        \n                            \n                                Donald Ogden Stewart, \n                            \n                        \n                            \n                                Waldo Salt',
    'Dec 1, 1940\n\xa0original',
    'May 16, 2000',
    '1h 52m',
    'Metro-Goldwyn-Mayer',
    'Mono'],
   'detail_crew': ['Cary Grant',
    'Katharine Hepburn',
    'James Stewart',
    'Ruth Hussey',
    'John Howard',
    'Roland Young',
    'John Halliday',
    'Virginia Weidler',
    'Mary Nash',
    'Henry Daniell',
    'Hillary Brooke',
    'George Cukor',
    'Philip Barry',
    'Donald Ogden Stewart',
    'Waldo Salt',
    'Joseph L. Mankiewicz',
    'Franz Waxman',
    'Joseph Ruttenberg',
    'Frank Sullivan',
    'Cedric Gibbons',
    'Edwin B. Willis',
    'Adrian']},
  'A Night at the Opera (1935)': {'detail_info': ['Comedy',
    'English',
    'Sam Wood',
    'Nov 15, 1935\n\xa0original',
    'May 4, 2004',
    '1h 33m',
    'Metro-Goldwyn-Mayer'],
   'detail_crew': ['Groucho Marx',
    'Harpo Marx',
    'Kitty Carlisle Hart',
    'Chico Marx',
    'Margaret Dumont',
    'Allan Jones',
    'Walter Woolf King',
    'Sig Ruman',
    'Sam Wood']}},
 'Top 100 Science Fiction & Fantasy Movies': {'The Wizard of Oz (1939)': {'detail_info': ['G',
    'Fantasy, \n                        \n                        Musical',
    'English',
    'Victor Fleming',
    'Victor Fleming, \n                            \n                        \n                            \n                                Mervyn LeRoy',
    'L. Frank Baum, \n                            \n                        \n                            \n                                Noel Langley, \n                            \n                        \n                            \n                                Florence Ryerson, \n                            \n                        \n                            \n                                Edgar Allan Woolf',
    'Aug 25, 1939\n\xa0wide',
    'Aug 12, 2003',
    '$5.5M',
    '1h 41m',
    'Metro-Goldwyn-Mayer',
    'Mono',
    'Flat (1.37:1)'],
   'detail_crew': ['Judy Garland',
    'Frank Morgan',
    'Ray Bolger',
    'Bert Lahr',
    'Jack Haley',
    'Billie Burke',
    'Margaret Hamilton',
    'Charley Grapewin',
    'Pat Walshe',
    'Clara Blandick',
    'Victor Fleming',
    'L. Frank Baum',
    'Noel Langley',
    'Florence Ryerson',
    'Edgar Allan Woolf',
    'Victor Fleming',
    'Mervyn LeRoy',
    'Harold Arlen',
    'Herbert Stothart',
    'Felix Mendelssohn',
    'Modest Mussorgsky',
    'Nikolai Rimsky-Korsakov',
    'Robert Schumann',
    'Harold Rosson',
    'Blanche Sewell',
    'Cedric Gibbons',
    'Edwin B. Willis',
    'Adrian',
    'Douglas Shearer']},
  'Black Panther (2018)': {'detail_info': ['PG-13 (Sequences of Action Violence|A Brief Rude Gesture)',
    'Adventure, \n                        \n                        Fantasy, \n                        \n                        Action',
    'English',
    'Ryan Coogler',
    'Kevin Feige',
    'Ryan Coogler, \n                            \n                        \n                            \n                                Joe Robert Cole',
    'Feb 16, 2018\n\xa0wide',
    'May 2, 2018',
    '$700.2M',
    '2h 14m',
    'Walt Disney',
    'DTS, Dolby Atmos',
    'Scope (2.35:1)',
    'Marvel Cinematic Universe'],
   'detail_crew': ['Chadwick Boseman',
    'Michael B. Jordan',
    "Lupita Nyong'o",
    'Danai Gurira',
    'Martin Freeman',
    'Daniel Kaluuya',
    'Letitia Wright',
    'Winston Duke',
    'Angela Bassett',
    'Forest Whitaker',
    'Andy Serkis',
    'Florence Kasumba',
    'Sterling K. Brown',
    'John Kani',
    'Ryan Coogler',
    'Ryan Coogler',
    'Joe Robert Cole',
    'Kevin Feige',
    "Louis D'Esposito",
    'Victoria Alonso',
    'Nate Moore',
    'Jeffrey Chernov',
    'Stan Lee',
    'Rachel Morrison',
    'Debbie Berman',
    'Michael P. Shawver',
    'Ludwig Göransson',
    'Hannah Beachler',
    'Ruth E. Carter',
    'Sarah Halley Finn']},
  'Avengers: Endgame (2019)': {'detail_info': ['PG-13 (Sequences of Sci-Fi Violence|Action|Some Language)',
    'Sci-fi, \n                        \n                        Adventure, \n                        \n                        Fantasy, \n                        \n                        Action',
    'English',
    'Anthony Russo, \n                            \n                        \n                            \n                                Joe Russo',
    'Kevin Feige',
    'Christopher Markus, \n                            \n                        \n                            \n                                Stephen McFeely',
    'Apr 26, 2019\n\xa0wide',
    'Jul 30, 2019',
    '$858.4M',
    '3h 1m',
    'Walt Disney',
    'Dolby Atmos, DTS, Dolby Digital, SDDS',
    'Scope (2.35:1)',
    'Marvel Cinematic Universe'],
   'detail_crew': ['Robert Downey Jr.',
    'Chris Evans',
    'Mark Ruffalo',
    'Chris Hemsworth',
    'Scarlett Johansson',
    'Jeremy Renner',
    'Brie Larson',
    'Paul Rudd',
    'Don Cheadle',
    'Karen Gillan',
    'Danai Gurira',
    'Bradley Cooper',
    'Gwyneth Paltrow',
    'Jon Favreau',
    'Benedict Wong',
    'Tessa Thompson',
    'Josh Brolin',
    'Tilda Swinton',
    'Robert Redford',
    'Dave Bautista',
    'Anthony Russo',
    'Joe Russo',
    'Christopher Markus',
    'Stephen McFeely',
    'Kevin Feige',
    "Louis D'Esposito",
    'Victoria Alonso',
    'Michael Grillo',
    'Trinh Tran',
    'Jon Favreau',
    'James Gunn',
    'Stan Lee',
    'Trent Opaloch',
    'Jeffrey Ford',
    'Matthew Schmidt',
    'Alan Silvestri',
    'Charles Wood',
    'Judianna Makovsky',
    'Sarah Halley Finn']},
  'Toy Story 4 (2019)': {'detail_info': ['G',
    'Comedy, \n                        \n                        Kids & family, \n                        \n                        Adventure, \n                        \n                        Fantasy, \n                        \n                        Animation',
    'English',
    'Josh Cooley',
    'Mark Nielsen, \n                            \n                        \n                            \n                                Jonas Rivera, \n                            \n                        \n                            \n                                Galyn Susman',
    'Andrew Stanton, \n                            \n                        \n                            \n                                Stephany Folsom',
    'Jun 21, 2019\n\xa0wide',
    'Oct 1, 2019',
    '$434.0M',
    '1h 40m',
    'Walt Disney',
    'DTS, Dolby Digital, Dolby Atmos',
    'Scope (2.35:1)',
    'Pixar'],
   'detail_crew': ['Tom Hanks',
    'Tim Allen',
    'Annie Potts',
    'Tony Hale',
    'Keegan-Michael Key',
    'Madeleine McGraw',
    'Christina Hendricks',
    'Jordan Peele',
    'Keanu Reeves',
    'Ally Maki',
    'Jay Hernandez',
    'Lori Alan',
    'Joan Cusack',
    'Bonnie Hunt',
    'Kristen Schaal',
    'Emily Davis',
    'Wallace Shawn',
    'John Ratzenberger',
    'Blake Clark',
    'June Squibb',
    'Josh Cooley',
    'Andrew Stanton',
    'Stephany Folsom',
    'Mark Nielsen',
    'Jonas Rivera',
    'Galyn Susman',
    'Andrew Stanton',
    'Lee Unkrich',
    'Pete Docter',
    'Patrick Lin',
    'Jean-Claude Kalache',
    'Axel Geddes',
    'Randy Newman',
    'Bob Pauley',
    'Kevin Reher',
    'Natalie Lyon']}},
 'Top 100 Special Interest Movies': {'The Bride of Frankenstein (1935)': {'detail_info': ['Horror',
    'English',
    'James Whale',
    'Carl Laemmle Jr.',
    'William Hurlbut, \n                            \n                        \n                            \n                                John L. Balderston, \n                            \n                        \n                            \n                                Mary Shelley',
    'Apr 22, 1935\n\xa0wide',
    'Aug 28, 2001',
    '1h 15m',
    'Realart Pictures Inc., \n                        \n                        Universal International Pictures, \n                        \n                        MCA/Universal Pictures [us], \n                        \n                        Universal Pictures, \n                        \n                        Film Classics Inc., \n                        \n                        Universal Home Entertainment',
    'Mono'],
   'detail_crew': ['Boris Karloff',
    'Elsa Lanchester',
    'Colin Clive',
    'Valerie Hobson',
    'O.P. Heggie',
    "Una O'Connor",
    'Ernest Thesiger',
    'Gavin Gordon',
    'Douglas Walton',
    'E.E. Clive',
    'Lucien Prival',
    'Dwight Frye',
    'Reginald Barlow',
    'Mary Gordon',
    'Anne Darling',
    'James Whale',
    'William Hurlbut',
    'John L. Balderston',
    'Mary Shelley',
    'Carl Laemmle Jr.',
    'Franz Schubert',
    'Franz Waxman',
    'John J. Mescall',
    'Ted J. Kent',
    'Charles D. Hall',
    'Jack P. Pierce']},
  'Frankenstein (1931)': {'detail_info': ['Horror',
    'English',
    'James Whale',
    'Carl Laemmle Jr.',
    'Mary Shelley, \n                            \n                        \n                            \n                                Peggy Webling, \n                            \n                        \n                            \n                                John L. Balderston, \n                            \n                        \n                            \n                                Francis Edward Faragoh',
    'Nov 21, 1931\n\xa0wide',
    'Aug 17, 1999',
    '1h 11m',
    'MCA/Universal Pictures [us], \n                        \n                        Universal Pictures, \n                        \n                        Screen Gems, \n                        \n                        Universal Home Entertainment, \n                        \n                        Realart Pictures Inc.',
    'Mono'],
   'detail_crew': ['Boris Karloff',
    'Colin Clive',
    'Mae Clarke',
    'John Boles',
    'Edward Van Sloan',
    'Dwight Frye',
    'Frederick Kerr',
    'Lionel Belmore',
    'Michael Mark',
    'Arletta Duncan',
    'Pauline Moore',
    'Francis Ford',
    'James Whale',
    'Mary Shelley',
    'Peggy Webling',
    'John L. Balderston',
    'Francis Edward Faragoh',
    'Carl Laemmle Jr.',
    'Bernhard Kaun',
    'Arthur Edeson',
    'Paul Ivano',
    'Clarence Kolster',
    'Charles D. Hall']},
  'Life Itself (2014)': {'detail_info': ['R (Language|Brief Sexual Images/Nudity)',
    'Documentary',
    'English',
    'Steve James',
    'Zak Piper, \n                            \n                        \n                            \n                                Steve James, \n                            \n                        \n                            \n                                Garrett Basch',
    'Jul 4, 2014\n\xa0limited',
    'Feb 17, 2015',
    '$809.7K',
    '2h 0m',
    'Magnolia Pictures'],
   'detail_crew': ['Roger Ebert',
    'Ava DuVernay',
    'Steve James',
    'Zak Piper',
    'Steve James',
    'Garrett Basch',
    'Martin Scorsese',
    'Steven Zaillian',
    'Michael W. Ferro Jr.',
    'Gordon Quinn',
    'Justine Nagan',
    'Kat White',
    'Mark Mitten',
    'Vinnie Malhotra',
    'Dana Kupper',
    'David E. Simpson',
    'Steve James',
    'Joshua Abrams']},
  'Man on Wire (2008)': {'detail_info': ['PG-13 (Some Sexuality|Some Nudity|Drug References)',
    'Documentary',
    'English',
    'James Marsh',
    'Simon Chinn',
    'Jul 25, 2008\n\xa0limited',
    'Dec 9, 2008',
    '$3.0M',
    '1h 30m',
    'Magnolia Pictures'],
   'detail_crew': ['Annie Allix',
    'Jean François Heckel',
    'James Marsh',
    'Simon Chinn',
    'Nick Fraser',
    'Andrea Meditch']}},
 'Top 100 Sports & Fitness Movies': {'Murderball (2005)': {'detail_info': ['R (Sexual Content|Language)',
    'Documentary',
    'English',
    'Henry Alex Rubin, \n                            \n                        \n                            \n                                Dana Adam Shapiro',
    'Dana Adam Shapiro',
    'Jul 8, 2005\n\xa0original',
    'Nov 29, 2005',
    '$1.5M',
    '1h 25m',
    'ThinkFilm'],
   'detail_crew': ['Henry Alex Rubin',
    'Dana Adam Shapiro',
    'Christian Ettinger',
    'Randy Manis',
    'Jeff Sackman',
    'Dana Adam Shapiro',
    'Mark Urman',
    'Henry Alex Rubin',
    "Conor O'Neill",
    'Geoffrey Richman',
    'Matthew Haasch',
    'David Egan']},
  'Athlete A (2020)': {'detail_info': ['PG-13 (Descriptions of Sexual Abuse|Mature Thematic Content)',
    'Documentary',
    'English',
    'Bonni Cohen, \n                            \n                        \n                            \n                                Jon Shenk',
    'Serin Marshall, \n                            \n                        \n                            \n                                Jen Sey, \n                            \n                        \n                            \n                                Julie Parker Benello',
    'Jun 25, 2020',
    '1h 43m'],
   'detail_crew': ['Maggie Nichols',
    'Jerry Moran',
    'Jen Sey',
    'Bonni Cohen',
    'Jon Shenk',
    'Richard Berge',
    'Dan Cogan',
    'Serin Marshall',
    'Jen Sey',
    'Julie Parker Benello',
    'Don Bernier',
    'Jeff Beal']},
  'Hoop Dreams (1994)': {'detail_info': ['PG-13',
    'Drama, \n                        \n                        Documentary',
    'English',
    'Steve James',
    'Peter Gilbert, \n                            \n                        \n                            \n                                Steve James, \n                            \n                        \n                            \n                                Frederick Marx',
    'Steve James, \n                            \n                        \n                            \n                                Frederick Marx',
    'Oct 14, 1994\n\xa0wide',
    'May 10, 2005',
    '$7.8M',
    '2h 54m',
    'New Line Home Video [us], \n                        \n                        Fine Line Features',
    'Stereo'],
   'detail_crew': ['Steve James',
    'William Gates',
    'Arthur Agee',
    'Sheila Agee',
    'Isiah Thomas',
    'Bill Gleason',
    'Steve James',
    'Steve James',
    'Frederick Marx',
    'Gordon Quinn',
    'Catherine Allan',
    'Peter Gilbert',
    'Steve James',
    'Frederick Marx',
    'Ben Sidran',
    'Peter Gilbert',
    'William Haugse']},
  'When We Were Kings (1996)': {'detail_info': ['PG',
    'Documentary',
    'English',
    'Leon Gast',
    'Taylor Hackford, \n                            \n                        \n                            \n                                Leon Gast',
    'Feb 14, 1997\n\xa0wide',
    'Dec 9, 2002',
    '$2.7M',
    '1h 24m',
    'Image Entertainment Inc., \n                        \n                        Gramercy Pictures, \n                        \n                        Polygram'],
   'detail_crew': ['Muhammad Ali',
    'George Foreman',
    'Don King',
    'B.B. King',
    'Mobutu Sese Seko',
    'Spike Lee',
    'Norman Mailer',
    'George Plimpton',
    'Thomas Hauser',
    'Malick Bowens',
    'Wilton Felder',
    'Wayne Henderson',
    'Stix Hooper',
    'Leon Gast',
    'David Sonenberg',
    'Taylor Hackford',
    'Leon Gast',
    'Miriam Makeba',
    'Maryse Alberti',
    'Paul Goldsmith',
    'Kevin Keating',
    'Albert Maysles',
    'Leon Gast',
    'Taylor Hackford',
    'Jeffrey Kusama-Hinte']}},
 'Top 100 Television Movies': {'Life Itself (2014)': {'detail_info': ['R (Language|Brief Sexual Images/Nudity)',
    'Documentary',
    'English',
    'Steve James',
    'Zak Piper, \n                            \n                        \n                            \n                                Steve James, \n                            \n                        \n                            \n                                Garrett Basch',
    'Jul 4, 2014\n\xa0limited',
    'Feb 17, 2015',
    '$809.7K',
    '2h 0m',
    'Magnolia Pictures'],
   'detail_crew': ['Roger Ebert',
    'Ava DuVernay',
    'Steve James',
    'Zak Piper',
    'Steve James',
    'Garrett Basch',
    'Martin Scorsese',
    'Steven Zaillian',
    'Michael W. Ferro Jr.',
    'Gordon Quinn',
    'Justine Nagan',
    'Kat White',
    'Mark Mitten',
    'Vinnie Malhotra',
    'Dana Kupper',
    'David E. Simpson',
    'Steve James',
    'Joshua Abrams']},
  'Afghan Star (2009)': {'detail_info': ['Documentary',
    'English',
    'Havana Marking',
    'Jun 26, 2009\n\xa0limited',
    'Mar 30, 2010',
    '$101.5K',
    '1h 28m',
    'Zeitgeist'],
   'detail_crew': ['Havana Marking']},
  'A Film Unfinished (2010)': {'detail_info': ['Documentary',
    'German',
    'Yael Hersonski',
    'Itai Ken-Tor, \n                            \n                        \n                            \n                                Noemi Schory',
    'Yael Hersonski',
    'Aug 18, 2010\n\xa0original',
    'Mar 8, 2011',
    '$311.5K',
    '1h 29m',
    'Oscilloscope Pictures'],
   'detail_crew': ['Yael Hersonski',
    'Yael Hersonski',
    'Itai Ken-Tor',
    'Noemi Schory',
    'Joel Alexis']},
  'Room 237 (2013)': {'detail_info': ['Documentary',
    'English',
    'Rodney Ascher',
    'Tim Kirk',
    'Mar 29, 2013\n\xa0limited',
    'Sep 24, 2013',
    '$259.8K',
    '1h 42m',
    'IFC Films'],
   'detail_crew': ['Jay Weidner',
    'Bill Blakemore',
    'Geoffrey Cocks',
    'Juli Kearns',
    'John Fell Ryan',
    'Rodney Ascher',
    'Tim Kirk']}},
 'Top 100 Western Movies': {'The Treasure of the Sierra Madre (1948)': {'detail_info': ['Drama, \n                        \n                        Western, \n                        \n                        Adventure',
    'English',
    'John Huston',
    'Henry Blanke',
    'B. Traven, \n                            \n                        \n                            \n                                John Huston',
    'Jan 6, 1948\n\xa0original',
    'Sep 30, 2003',
    '2h 4m',
    'Warner Bros. Pictures, \n                        \n                        MGM/UA Home Entertainment Inc.'],
   'detail_crew': ['Humphrey Bogart',
    'Walter Huston',
    'Tim Holt',
    'Bruce Bennett',
    'Barton MacLane',
    'Alfonso Bedoya',
    'John Huston',
    'Jack L. Warner',
    'Henry Blanke',
    'Buddy Kaye',
    'Max Steiner',
    'Ted D. McCord',
    'Owen Marks',
    'John Hughes',
    'Richard Maybery',
    'B. Traven',
    'John Huston']},
  'The Good, the Bad and the Ugly (1966)': {'detail_info': ['R',
    'Western',
    'English',
    'Sergio Leone',
    'Alberto Grimaldi',
    'Agenore Incrocci, \n                            \n                        \n                            \n                                Furio Scarpelli, \n                            \n                        \n                            \n                                Luciano Vincenzoni, \n                            \n                        \n                            \n                                Sergio Leone',
    'Dec 20, 1967\n\xa0original',
    'Nov 7, 2006',
    '2h 41m',
    'Metro-Goldwyn-Mayer, \n                        \n                        United Artists'],
   'detail_crew': ['Clint Eastwood',
    'Eli Wallach',
    'Lee Van Cleef',
    'Aldo Giuffré',
    'Chelo Alonso',
    'Mario Brega',
    'Luigi Pistilli',
    'Rada Rassimov',
    'Sergio Leone',
    'Agenore Incrocci',
    'Furio Scarpelli',
    'Luciano Vincenzoni',
    'Sergio Leone',
    'Alberto Grimaldi',
    'Ennio Morricone',
    'Tonino Delli Colli',
    'Eugenio Alabiso',
    'Fabrizio Gianni',
    'Giancarlo Santi']},
  'Stagecoach (1939)': {'detail_info': ['Western',
    'English',
    'John Ford',
    'John Ford',
    'Ernest Haycox, \n                            \n                        \n                            \n                                Dudley Nichols, \n                            \n                        \n                            \n                                Ben Hecht',
    'Mar 2, 1939\n\xa0wide',
    'Sep 3, 2002',
    '1h 36m',
    'United Artists, \n                        \n                        UCLA Film and Television Archive [us], \n                        \n                        Warner Home Vídeo',
    'Flat (1.37:1)'],
   'detail_crew': ['John Wayne',
    'Claire Trevor',
    'Andy Devine',
    'John Carradine',
    'Thomas Mitchell',
    'Louise Platt',
    'George Bancroft',
    'Donald Meek',
    'Berton Churchill',
    'Tim Holt',
    'Tom Tyler',
    'Chris-Pin Martin',
    'Francis Ford',
    'Jack Pennick',
    'John Ford',
    'Ernest Haycox',
    'Dudley Nichols',
    'Ben Hecht',
    'Walter Wanger',
    'John Ford',
    'Gerard Carbonara',
    'Bert Glennon',
    'Otho Lovering',
    'Dorothy Spencer',
    'Walter Reynolds',
    'Alexander Toluboff',
    'Walter Plunkett',
    'Daniel Keefe',
    'Jack Kirston',
    'David Sharpe',
    'Ray Binger',
    'Frank Maher',
    'Wiard Ihnen']},
  'High Noon (1952)': {'detail_info': ['PG (Some Western Violence|Smoking)',
    'Western',
    'English',
    'Fred Zinnemann',
    'Stanley Kramer',
    'John W. Cunningham, \n                            \n                        \n                            \n                                Carl Foreman',
    'Jul 24, 1952\n\xa0original',
    'Oct 22, 2002',
    '1h 25m',
    'Criterion Collection, \n                        \n                        Republic Pictures, \n                        \n                        United Artists'],
   'detail_crew': ['Gary Cooper',
    'Grace Kelly',
    'Katy Jurado',
    'Thomas Mitchell',
    'Lloyd Bridges',
    'Otto Kruger',
    'Lon Chaney Jr.',
    'Harry Morgan',
    'Lee Van Cleef',
    'Robert J. Wilke',
    'Sheb Wooley',
    'Ian MacDonald',
    'Fred Zinnemann',
    'John W. Cunningham',
    'Carl Foreman',
    'Stanley Kramer',
    'Dimitri Tiomkin',
    'Floyd Crosby',
    'Elmo Williams',
    'Rudolph Sternad',
    'Ben Hayne',
    'Murray Waite']}}}

In [17]:
import pandas as pd
columns = ['Rating', 'Genre', 'Original Language', 'Director', 'Producer', 'Writer', 'Release Date(Theaters)', 'Release Date (Streaming)', 'Box Office (Gross USA)', 'Runtime', 'Distributor', 'Sound Mix', 'Aspect Ratio', 'View the collection']

container = []
for key, value_dict in movies_dict.items():
    sheet_title = 'df_' + key.replace(' ', '_').replace('&','and')
    for key, value in value_dict.items():
        movie_title = key
        movie_info = value
        context = [variable.replace('\n','').replace('  ','').replace('\xa0',' ') for variable in movie_info['detail_info']]

        
    aaa = pd.DataFrame.from_dict(data=context)
    container.append(aaa)

In [18]:
container[3]

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,G,Comedy,English,Charlie Chaplin,Charlie Chaplin,Charlie Chaplin,"Feb 5, 1936 wide","Nov 16, 2010",$163.2K,1h 29m,United Artists,Mono,Flat (1.37:1)


In [3]:
target_website = Website('aaa', 'top_movies', 'https://www.rottentomatoes.com', 'https://www.rottentomatoes.com/top/bestofrt/top_100_action__adventure_movies/')

contents = [
    WebElement('text(detail_info)', 'css', '#mainColumn > section.panel.panel-rt.panel-box.movie_info.media > div > div > ul > li > div.meta-value'),
    WebElement('text(detail_crew)', 'css', '#movie-cast > div > div > div > div.media-body > a > span')
]
iteration = WebElement('iterations','css','#top_movies_main > div > table > tbody > tr > td:nth-child(3) > a')
iteration_actions_in = [
    WebElement('click','',''),
    WebElement('load','','')
]
iteration_actions_out = [
    WebElement('back','',''),
    WebElement('time','','')
]
movies_top_100_dict = crawler.getContentsInSequence(target_website, contents, iteration, iteration_actions_in, iteration_actions_out)
movies_top_100_dict


checkWebdriverVersion : already the latest chromedriver


{'Black Panther (2018)': {'detail_info': ['PG-13 (Sequences of Action Violence|A Brief Rude Gesture)',
   'Adventure, \n                        \n                        Fantasy, \n                        \n                        Action',
   'English',
   'Ryan Coogler',
   'Kevin Feige',
   'Ryan Coogler, \n                            \n                        \n                            \n                                Joe Robert Cole',
   'Feb 16, 2018\n\xa0wide',
   'May 2, 2018',
   '$700.2M',
   '2h 14m',
   'Walt Disney',
   'DTS, Dolby Atmos',
   'Scope (2.35:1)',
   'Marvel Cinematic Universe'],
  'detail_crew': ['Chadwick Boseman',
   'Michael B. Jordan',
   "Lupita Nyong'o",
   'Danai Gurira',
   'Martin Freeman',
   'Daniel Kaluuya',
   'Letitia Wright',
   'Winston Duke',
   'Angela Bassett',
   'Forest Whitaker',
   'Andy Serkis',
   'Florence Kasumba',
   'Sterling K. Brown',
   'John Kani',
   'Ryan Coogler',
   'Ryan Coogler',
   'Joe Robert Cole',
   'Kevin Feige'

In [16]:
movies_top_100_dict = {}
for i, element in enumerate(genre_top_100_crawled['genre_urls']):
    target_url = rotten_tomatoes.main_url + element
    target_website = Website(genre_top_100_crawled['genre_names'][i], 'top_movies', 'https://www.rottentomatoes.com', target_url)
    contents = [
        WebElement('text(movie_names)', 'css', '#top_movies_main > div > table > tbody > tr > td:nth-child(3) > a'),
        WebElement('url(movie_urls)', 'css', '#top_movies_main > div > table > tbody > tr > td:nth-child(3) > a'),
        WebElement('table(movie_table)', 'css', '#top_movies_main > div > table')
    ]
    movies_top_100_crawled = crawler.getContents(target_website, contents, module='selenium')
    movies_top_100_dict[target_website.name] = movies_top_100_crawled
    print(f'{i} {target_website.name} Done!                                   ', end='\r')


print()
for key, value in movies_top_100_dict.items():
    print(key)
    print(*value, sep='\n')
    print()


checkWebdriverVersion : already the latest chromedriver
0 Top 100 Action & Adventure Movies Done!                      

checkWebdriverVersion : already the latest chromedriver
1 Top 100 Animation Movies Done!                      

checkWebdriverVersion : already the latest chromedriver
2 Top 100 Art House & International Movies Done!                      

checkWebdriverVersion : already the latest chromedriver
3 Top 100 Classics Movies Done!                      

checkWebdriverVersion : already the latest chromedriver
4 Top 100 Comedy Movies Done!                      

checkWebdriverVersion : already the latest chromedriver
5 Top 100 Documentary Movies Done!                      

checkWebdriverVersion : already the latest chromedriver
6 Top 100 Drama Movies Done!                      

checkWebdriverVersion : already the latest chromedriver
7 Top 100 Horror Movies Done!                      

checkWebdriverVersion : already the latest chromedriver
8 Top 100 Kids & Family Movies 

In [49]:
details_dict = {}
for i, element in enumerate(movies_top_100_dict['Top 100 Action & Adventure Movies']['movie_urls'][0:5]):
    target_url = rotten_tomatoes.main_url + element
    target_website = Website(movies_top_100_dict['Top 100 Action & Adventure Movies']['movie_names'][i], 'movie_details', 'https://www.rottentomatoes.com', target_url)
    contents = [
        WebElement('text(TOMATOMETER)', 'css', 'div > span.percentage'),
        WebElement('text(AUDIENCE SCORE)', 'css', 'div > span.percentage'),
        WebElement('text(detail_info)', 'css', '#mainColumn > section.panel.panel-rt.panel-box.movie_info.media > div > div > ul > li > div.meta-value'),
        WebElement('text(detail_crew)', 'css', '#movie-cast > div > div > div > div.media-body > a > span'),
    ]
    details_crawled = crawler.getContents(target_website, contents, module='selenium')
    details_dict[target_website.name] = details_crawled
    print(f'{i+1} {target_website.name} Done!                                 ', end='\r')


checkWebdriverVersion : already the latest chromedriver
0 Black Panther (2018) Done!                                 
checkWebdriverVersion : already the latest chromedriver
1 Avengers: Endgame (2019) Done!                                 
checkWebdriverVersion : already the latest chromedriver
2 Mission: Impossible - Fallout (2018) Done!                                 
checkWebdriverVersion : already the latest chromedriver
3 Mad Max: Fury Road (2015) Done!                                 
checkWebdriverVersion : already the latest chromedriver


In [87]:
genre_movies_dict = {}
for key, value in movies_top_100_dict.items():
    details_dict = {}
    for i, element in enumerate(value['movie_urls'][0:3]):
        target_url = rotten_tomatoes.main_url + element
        target_website = Website(value['movie_names'][i], 'movie_details', 'https://www.rottentomatoes.com', target_url)
        contents = [
            WebElement('text(detail_info)', 'css', '#mainColumn > section.panel.panel-rt.panel-box.movie_info.media > div > div > ul > li > div.meta-value'),
            WebElement('text(detail_crew)', 'css', '#movie-cast > div > div > div > div.media-body > a > span')
        ]
        details_crawled = crawler.getContents(target_website, contents, module='selenium')
        details_dict[target_website.name] = details_crawled
        print(f'{i+1} {target_website.name} Done!                                 ', end='\r')
    for key in details_dict.keys():
        detail_info = [element.replace('\n','').replace('  ','').replace('\xa0',' ') for element in details_dict[key]['detail_info']]
        details_dict[key]['detail_info'] = detail_info
    context = [details_dict[i]['detail_info'] for i in details_dict.keys()]
    df = pd.DataFrame(data=context, index=details_dict.keys(), columns=columns)
    genre_movies_dict[key] = df


checkWebdriverVersion : already the latest chromedriver
1 Black Panther (2018) Done!                                 
checkWebdriverVersion : already the latest chromedriver
2 Avengers: Endgame (2019) Done!                                 
checkWebdriverVersion : already the latest chromedriver
3 Mission: Impossible - Fallout (2018) Done!                                 
checkWebdriverVersion : already the latest chromedriver
1 Toy Story 4 (2019) Done!                                 
checkWebdriverVersion : already the latest chromedriver
2 Spider-Man: Into the Spider-Verse (2018) Done!                                 
checkWebdriverVersion : already the latest chromedriver
3 Inside Out (2015) Done!                                 
checkWebdriverVersion : already the latest chromedriver
1 Parasite (Gisaengchung) (2019) Done!                                 
checkWebdriverVersion : already the latest chromedriver
2 The Cabinet of Dr. Caligari (Das Cabinet des Dr. Caligari) (1920) Done

ValueError: 14 columns passed, passed data had 13 columns

In [76]:
columns = ['Rating', 'Genre', 'Original Language', 'Director', 'Producer', 'Writer', 'Release Date(Theaters)', 'Release Date (Streaming)', 'Box Office (Gross USA)', 'Runtime', 'Distributor', 'Sound Mix', 'Aspect Ratio', 'View the collection']
for key in details_dict.keys():
    detail_info = [element.replace('\n','').replace('  ','').replace('\xa0',' ') for element in details_dict[key]['detail_info']]
    details_dict[i]['detail_info'] = detail_info
context = [details_dict[i]['detail_info'] for i in details_dict.keys()]

In [79]:
df = pd.DataFrame(data=context, index=details_dict.keys(), columns=columns)
df

,Rating,Genre,Original Language,Director,Producer,Writer,Release Date(Theaters),Release Date (Streaming),Box Office (Gross USA),Runtime,Distributor,Sound Mix,Aspect Ratio,View the collection
Black Panther (2018),PG-13 (Sequences of Action Violence|A Brief Ru...,"Adventure, Fantasy, Action",English,Ryan Coogler,Kevin Feige,"Ryan Coogler, Joe Robert Cole","Feb 16, 2018 wide","May 2, 2018",$700.2M,2h 14m,Walt Disney,"DTS, Dolby Atmos",Scope (2.35:1),Marvel Cinematic Universe
Avengers: Endgame (2019),PG-13 (Sequences of Sci-Fi Violence|Action|Som...,"Sci-fi, Adventure, Fantasy, Action",English,"Anthony Russo, Joe Russo",Kevin Feige,"Christopher Markus, Stephen McFeely","Apr 26, 2019 wide","Jul 30, 2019",$858.4M,3h 1m,Walt Disney,"Dolby Atmos, DTS, Dolby Digital, SDDS",Scope (2.35:1),Marvel Cinematic Universe
Mission: Impossible - Fallout (2018),PG-13 (Intense Sequences of Action|Brief Stron...,"Mystery & thriller, Adventure, Action",English,Christopher McQuarrie,"Tom Cruise, Christopher McQuarrie, Jake Myers,...",Christopher McQuarrie,"Jul 27, 2018 wide","Oct 23, 2018",$220.1M,2h 27m,Paramount Pictures,"DTS, Dolby Atmos, Dolby Digital",Scope (2.35:1),None
Mad Max: Fury Road (2015),R (Intense Sequences of Violence|Disturbing Im...,"Adventure, Action",English,George Miller,"Doug Mitchell, George Miller, P.J. Voeten","George Miller, Brendan McCarthy, Nick Lathouris","May 15, 2015 wide","Sep 1, 2015",$153.6M,2h 0m,Warner Bros. Pictures,"Dolby Atmos, Dolby Digital, SDDS, Datasat",Scope (2.35:1),None
Spider-Man: Into the Spider-Verse (2018),PG (Mild Language|Frenetic Action Violence|The...,"Comedy, Kids & family, Adventure, Fantasy, Ani...",English,"Bob Persichetti, Peter Ramsey, Rodney Rothman","Avi Arad, Amy Pascal, Phil Lord, Chris Miller,...","Phil Lord, Rodney Rothman","Dec 14, 2018 wide","Mar 19, 2019",$190.2M,1h 57m,"Sony Pictures Entertainment, Columbia Pictures","Dolby Atmos, DTS, Dolby Digital, SDDS",Scope (2.35:1),None


In [80]:
aaa = {'a':'bb'}

In [82]:
for key, value in aaa.items():
    print(key)
    print(value)

a
bb


In [4]:
aaa = f'#top_movies_main > div > table > tbody > tr:nth-child({i}) > td:nth-child(3)'
for i in range(10):
    print(aaa)

NameError: name 'i' is not defined